---
# SURFER v4.0
---
__Author__ : Laridon Amaury 

__Content__ : Extended version of SURFER v2.0 based on Monteiro (2022) and SURFER v3.0 based on Victor Couplet

The version v4.0 is done by Amaury Laridon for his Master's Thesis.

## What's new in the version v4.0 ? 

- AMOC/GIS Cascade calibration module 
- Representation of the impacts on the carbon cycle of an AMOC collapse

---
---

#### Library

In [165]:
### Import Library ### 

import numpy as np
from math import floor,log10
from scipy.integrate import solve_ivp, odeint
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy import linalg
from scipy.sparse import diags
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
%matplotlib nbagg

### Data and Figure Path ### 

save_fig_path = "/home/amaury/Bureau/LPHYS2299 - Master's Thesis/Code/SURFER/SURFER v4.0/Figure/"
save_fig_path_color = "/home/amaury/Bureau/LPHYS2299 - Master's Thesis/Code/SURFER/SURFER v4.0/Figure/Colors/"
save_fig_path_scenarios = "/home/amaury/Bureau/LPHYS2299 - Master's Thesis/Code/SURFER/SURFER v4.0/Figure/Scenarios/"
data_path = "/home/amaury/Bureau/LPHYS2299 - Master's Thesis/Code/SURFER/SURFER v4.0/Data/"

#### Colormap

In [166]:
# Color Interpolator
# Given a sequence of N >= 2 colors ("the source"), 
# this script produces a sequence of any M >= 2 colors ("the target")
# with the first and last colors of the target identical to the source,
# and the colors in between equally spaced

# Author - Francois Massonnet (francois.massonnet@uclouvain.be / @FMassonnet)
# Date   - 12 Dec 2022

import matplotlib.pyplot as plt
import numpy as np

def colInterpolatOr(sourceColors, nTarget, colorCode = "RGB"):

  """ - sourceColors is a tuple of colors, by default in RGB format
      - nTarget is the desired number of output colors
      - colorCode : the format, either "RGB" (0-1) or "HEX"
  """

  if colorCode == "HEX":
    # From https://stackoverflow.com/questions/29643352/converting-hex-to-rgb-value-in-python
    sourceColors = [tuple(int(s[i:i + 2], 16) / 255. for i in (1, 3, 5)) for s in sourceColors] 
  elif colorCode != "RGB":
    print("ERROR: color code unknown")
  
  nSource = len(sourceColors)

  # List of output colors
  targetColors = list()


  # Let alpha be a scalar between 0 and 1, where 0 = first source color 
  # and 1 = last source color

  for alpha in np.linspace(0, 1, nTarget):

    # Identify the pair of source colors that will be used 
    # to make the linear interpolation. The variable "iRef" is the 
    # index of the first of these two source colors (iRef is between 0
    # and nSource - 1)

    iRef = int(np.floor(alpha * (nSource - 1)))

    # Reset last index to nSource - 1 since the last color is built 
    # from the last line
    if iRef == nSource - 1:
      iRef -= 1

    # Parameterize the corresponding line and fetch the value at alpha
    # beta is the parameter defining where (0 - 1) we are in between the two
    # source colors involved in the interpolation
    # beta is a simple scaling of alpha for the running interval

    beta = alpha * (nSource - 1) - iRef

    # Build the RGB triplet
    r = sourceColors[iRef][0] + beta * (sourceColors[iRef + 1][0] - sourceColors[iRef][0])
    g = sourceColors[iRef][1] + beta * (sourceColors[iRef + 1][1] - sourceColors[iRef][1])
    b = sourceColors[iRef][2] + beta * (sourceColors[iRef + 1][2] - sourceColors[iRef][2])


    
    if colorCode == "HEX":
      outputColor = '#%02x%02x%02x' % (int(r * 255), int(g * 255), int(b * 255))
    else:   
      outputColor = [r, g, b]

    targetColors.append(outputColor)
  

  return targetColors

def example():
  # List of colors that we want to pass through
  sourceColors = ["#0077BB", "#CC3311",]
  nSource = len(sourceColors)


  # Number of colors desired
  nTargets = [2, 16, 30]

  fig, ax = plt.subplots(len(nTargets) + 1, 1, figsize = (6, 6))

  for j in np.arange(nSource):
    xStart = j / nSource
    xEnd   = (j + 1) / nSource

    ax[0].fill_between((xStart, xEnd), (1, 1), color = sourceColors[j], edgecolor = "white")
    ax[0].set_title("Original colors (" + str(nSource) + ")")

  for f, n in enumerate(nTargets):
    targetColors = colInterpolatOr(sourceColors, colorCode = "HEX", nTarget = n)
  
    for j in np.arange(n):
      xStart = j / n
      xEnd   = (j + 1) / n

      ax[f + 1].fill_between((xStart, xEnd), (1, 1), color = targetColors[j], edgecolor = "white")
      ax[f + 1].set_title("Interpolated colors (" + str(n) + ")")


  for a in ax.flatten():
    a.set_xticklabels("")
    a.set_yticklabels("")
    a.set_xticks([])
    a.set_yticks([])
  plt.suptitle("https://github.com/fmassonn/snippets.git")
  

  figOut = save_fig_path_color+"colInterpolatOr.png"
  fig.tight_layout()
  fig.savefig(figOut, dpi = 300)
  print(figOut + " printed")


if __name__ == '__main__':
  example()

<IPython.core.display.Javascript object>

/home/amaury/Bureau/LPHYS2299 - Master's Thesis/Code/SURFER/SURFER v4.0/Figure/Colors/colInterpolatOr.png printed


---
# Model SURFER v4.0
---

## 1. Dynamical Model

In [167]:
mA=1.727*10**20
mO=7.8*10**22
carbon_molar_mass=12*10**-3
water_molar_mass=18*10**-3
CO23_sat = 68.27

fco2 = 3.9
beta = 1.1143
gamma = 0.8357
cvol = 0.13

tau_ch4 = 9.5

Sglpot = 0.5
Sglxi = 2
taugl = 200

def SURFER_T(co2_fossil,co2_landuse,ch4_fossil,ch4_landuse,y0,t0,tend,dt,**kwargs):

    if kwargs['ktemp'] == True :
        ktemp = 1
    else : ktemp = 0
        
    if kwargs['sediments'] == True : 
        sed = 1
    else : sed = 0
        
    if kwargs['weathering'] == True : 
        weathering = 1
    else : weathering = 0

    # Arrays to store the carbon flux coefficients computed at each iteration #
    k_AU_list = []
    k_UI_list = []
    k_IU_list = []
    k_ID_list = []
    k_DI_list = []
    # Arrays to store the freshwater fluxes computed at each iteration #
    f_gis_list = []
    f_w_list = []
    f_ep_list = []
    f_o_list = []

    def f(t,y) :
    
        K0U= k0(T_U+ktemp*y[9],S_U)
        K1U= k1(T_U+ktemp*y[9],S_U)
        K2U= k2(T_U+ktemp*y[9],S_U)
        BU = b(y[2],y[5],K1U,K2U)

        K1D = k1(T_D+ktemp*y[11],S_D)
        K2D = k2(T_D+ktemp*y[11],S_D)
        CO32_D = CO32D(y[4],y[7],K1D,K2D)

        co2_em_landuse = co2_landuse(t)
        co2_em_fossil = co2_fossil(t)
        ch4_em_landuse = ch4_landuse(t) 
        ch4_em_fossil = ch4_fossil(t) 
        
        E = co2_em_landuse + co2_em_fossil
        F_volcanism = V*sed
        F_CaCO3 = weathering*F_CaCO3_pi*(1+0.049*y[9])+(1-weathering)*F_CaCO3_pi
        F_CaSiO3 = weathering*F_CaSiO3_pi*np.exp(0.09*y[9])+(1-weathering)*F_CaSiO3_pi
        F_weathering = sed*((1-X)*F_CaCO3+(2-X)*F_CaSiO3)
        F_river = sed*((2-X)*(F_CaCO3+F_CaSiO3))+F_LR
        F_alk_river = sed*(2*(F_CaCO3+F_CaSiO3))
        
        if kwargs['atmtoland_flux'] == 'old' : 
            F_AL = K_AL_pi*(beta_L*MA_pi*(1-MA_pi/y[0])-(y[1]-y[13])) + F_LR
        else : F_AL = K_AL_pi*(beta_L*(y[0]-MA_pi)/np.exp(k_al_exp*beta*np.log(2)/fco2*y[9])-(y[1]-y[13])) + F_LR

        if amoc_impact_carb==False:
            F_AU = K_AU_pi*(y[0]-mA/(WU*K0U)*BU*y[2])
            F_UI = K_UI_pi*(y[2]-MU_pi/MI_pi*y[3])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[3]/MI_pi
            F_ID = K_ID_pi*(y[3]-MI_pi/MD_pi*y[4])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[4]/MD_pi
            k_AU_t = K_AU_pi
            k_UI_t = K_UI_pi
            k_IU_t = K_IU_pi
            k_ID_t = K_ID_pi
            k_DI_t = K_DI_pi
            k_AU_list.append(k_AU_t)
            k_UI_list.append(k_UI_t)
            k_IU_list.append(k_IU_t)
            k_ID_list.append(k_ID_t)
            k_DI_list.append(k_DI_t)
            
        if amoc_impact_carb==True:
            if cGenie_calib==True:
                psi_0 = 0.9284
            else:
                psi_O = 1

            if amoc_impact_carb_au==True:
                k_AU_t = K_AU_pi+((c_AU*K_AU_pi)*(psi_0-y[16]))
                k_AU_list.append(k_AU_t)
                F_AU = k_AU_t*(y[0]-mA/(WU*K0U)*BU*y[2])
        
            if amoc_impact_carb_au==False:
                k_AU_t = K_AU_pi
                k_AU_list.append(k_AU_t)
                F_AU = k_AU_t*(y[0]-mA/(WU*K0U)*BU*y[2])
                
            if impact_k_ui==True and impact_k_iu==True:
                k_UI_t = K_UI_pi+(c_UI*K_UI_pi*(psi_0-y[16]))
                k_UI_list.append(k_UI_t)
                k_IU_t = K_IU_pi+(c_IU*K_IU_pi*(psi_0-y[16]))
                k_IU_list.append(k_IU_t)

                F_UI = k_UI_t*y[2]-k_IU_t*y[3]+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[3]/MI_pi
                #F_UI = K_UI_pi*(y[2]-MU_pi/MI_pi*y[3])+K_UI_pi*(psi_0-y[16])*(c_UI*y[2]-c_IU*y[3])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[3]/MI_pi

            if impact_k_ui==True and impact_k_iu==False:
                k_UI_t = K_UI_pi+(c_UI*K_UI_pi*(psi_0-y[16]))
                k_UI_list.append(k_UI_t)
                k_IU_t = K_IU_pi
                k_IU_list.append(k_IU_t)

                F_UI = k_UI_t*y[2]-k_IU_t*y[3]+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[3]/MI_pi
                #F_UI = K_UI_pi*(y[2]-MU_pi/MI_pi*y[3])+K_UI_pi*(psi_0-y[16])*(c_UI*y[2])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[3]/MI_pi

            if impact_k_ui==False and impact_k_iu==True:
                k_UI_t = K_UI_pi
                k_UI_list.append(k_UI_t)
                k_IU_t = K_IU_pi+(c_IU*K_IU_pi*(psi_0-y[16]))
                k_IU_list.append(k_IU_t)

                F_UI = k_UI_t*y[2]-k_IU_t*y[3]+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[3]/MI_pi
                #F_UI = K_UI_pi*(y[2]-MU_pi/MI_pi*y[3])-K_UI_pi*(psi_0-y[16])*(c_IU*y[3])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[3]/MI_pi
            
            if impact_k_ui==False and impact_k_iu==False:
                k_UI_t = K_UI_pi
                k_UI_list.append(k_UI_t)
                k_IU_t = K_IU_pi
                k_IU_list.append(k_IU_t)

                F_UI = k_UI_t*y[2]-k_IU_t*y[3]+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[3]/MI_pi
                #F_UI = K_UI_pi*(y[2]-MU_pi/MI_pi*y[3])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[3]/MI_pi

            if amoc_impact_carb_id==True:
                if impact_k_id==True and impact_k_di==True:
                    k_ID_t = K_ID_pi+(c_ID*K_UI_pi*(psi_0-y[16]))
                    k_ID_list.append(k_ID_t)
                    k_DI_t = K_DI_pi+(c_DI*K_DI_pi*(psi_0-y[16]))
                    k_DI_list.append(k_DI_t)

                    F_ID = k_ID_t*y[3]-k_DI_t*y[4]+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[4]/MD_pi
                    #F_ID = K_ID_pi*(y[3]-MI_pi/MD_pi*y[4])+K_ID_pi*(psi_0-y[16])*(c_ID*y[3]-c_DI*y[4])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[4]/MD_pi

                if impact_k_id==True and impact_k_di==False:
                    k_ID_t = K_ID_pi+(c_ID*K_UI_pi*(psi_0-y[16]))
                    k_ID_list.append(k_ID_t)
                    k_DI_t = K_DI_pi
                    k_DI_list.append(k_DI_t)

                    F_ID = k_ID_t*y[3]-k_DI_t*y[4]+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[4]/MD_pi
                    #F_ID = K_ID_pi*(y[3]-MI_pi/MD_pi*y[4])+K_ID_pi*(psi_0-y[16])*(c_ID*y[3])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[4]/MD_pi

                if impact_k_id==False and impact_k_di==True:
                    k_ID_t = K_ID_pi
                    k_ID_list.append(k_ID_t)
                    k_DI_t = K_DI_pi+(c_DI*K_DI_pi*(psi_0-y[16]))
                    k_DI_list.append(k_DI_t)

                    F_ID = k_ID_t*y[3]-k_DI_t*y[4]+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[4]/MD_pi
                    #F_ID = K_ID_pi*(y[3]-MI_pi/MD_pi*y[4])-K_ID_pi*(psi_0-y[16])*(c_DI*y[4])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[4]/MD_pi   
            if amoc_impact_carb_id==False:
                k_ID_t = K_ID_pi
                k_ID_list.append(k_ID_t)
                k_DI_t = K_DI_pi
                k_DI_list.append(k_DI_t)
                
                F_ID = K_ID_pi*(y[3]-MI_pi/MD_pi*y[4])+sed*(F_CaCO3_pi + F_CaSiO3_pi)*y[4]/MD_pi

        
        F_accumulation = sed*(acc(CO32_D,y[8]/2))
        F_burial = sed*(bur(CO32_D,y[8]/2))
        F_alk_UI = sed*(K_alk_UI*(y[5]-QU_pi/QI_pi*y[6]) +2*(F_CaCO3_pi + F_CaSiO3_pi)*y[6]/QI_pi)
        F_alk_ID = sed*(K_alk_ID*(y[6]-QI_pi/QD_pi*y[7]) +2*(F_CaCO3_pi + F_CaSiO3_pi)*y[7]/QD_pi)
        F_alk_accumulation = sed*(2*F_accumulation)
        
        if y[8] > 0 : 
            allow_acc = 1
        else :
            if F_accumulation > 0 :
                allow_acc = 1
            else :
                allow_acc = 0

        ### Tipping Points Dynamics ###
        if AGTCCM==False:

            ## Initialization of vectors ## 
            COUPLING = S_moy@(1-y[15:23].T) # Matrix with the coupling coefficients
            # The right term, which is a vector (8x1), measure how much the different Tipping Element are collasped while the left term, which is a matrix (8x8) measure
            # The different coupling coefficients between the Tipping Elements.
            # The vector COUPLING which is (8x1)
            H = -y[15:23]**3+A2*y[15:23]**2+A1*y[15:23]+C1*y[9]-d*COUPLING+C0 # Matrix encoding the tipping elements dynamics
            MU = H*0 # Initialization of the mu coefficient encoding the characteristic time scale for each tipping element
            
            ## Definition of the mu timescales regarding the state of the tipping element ## 
            for i in range(0,len(H)) :
                # If the add element state is positive and the tipping element state is lower than 1 
                # then mu is defined as the positive characteristic time scale of the tipping element but 
                # if the tipping element state is already higher than 1 the mu is set to zero. 
                if H[i]>0 : 
                    if y[15+i]< 1 :
                        MU[i] = 1/TAUP[i]
                    else :
                        MU[i] = 0
                else :
                # If the add element state is negative and the tipping element state is higher than 0.01
                # then mu is defined as the negative characteristic time scale of the tipping element but
                # if the tipping element state is between 0 and 0.01 then mu is set to zero. 
                # if the tipping element is state is negative a negative time scale based on the negative characteristic
                # time scale is set to positively define again the tipping element state. 
                    if y[15+i]> 0.01 :
                        MU[i] = 1/TAUM[i]
                    elif 0 < y[15+i]< 0.01 :
                        MU[i] = 0
                    else : MU[i] = -1/TAUM[i]
                        
            ## add coupling for AMOC (need derivatives)
            
            if adv_param_amoc_vic == True:
                H[1] = H[1] + d*(c_10*MU[0]*H[0]+c_12*MU[2]*H[2]+c_13*MU[3]*H[3])
            
            if H[1]>0 : 
                if y[15+1]< 1:
                    MU[1] = 1/TAUP[1]
                else : MU[1] = 0
            else :
                if y[15+1]> 0.01 :
                    MU[1] = 1/TAUM[1]
                elif 0< y[15+1]<0.01 :
                    MU[1] = 0
                else :
                    MU[1] = -1/TAUM[1]
            
            ## Computation of the consequences on CO2 and CH4 of the tipping element states ## 
            co2_em_tps = cfeedback*CPOT@(MU*H)*-1
            ch4_em_tps = cfeedback*CH4POT@(MU*H)*-1

        if AGTCCM==True:

                                        ### Computing coupling matrix ###
            COUPLING_AGTCCM = S_moy_AGTCCM@(1-y[15:23].T)

                                    ### Other Tipping Elements Dynamic ### 
            ## Computation of the dynamics of all other Tipping Elements with the old parameterization ##
            OTE_H = -y[17:23]**3+A2_AGTCCM*y[17:23]**2+A1_AGTCCM*y[17:23]+C1_AGTCCM*y[9]+C0_AGTCCM - d*COUPLING_AGTCCM[2:]

                        ### New dynamical modelisation of AMOC and GIS state and coupling ### 

                                             ## GIS Dynamic ## 
            # Computation of the dynamics of GIS and AMOC based on the new parameterization #
            dGIS = -y[15]**3+a_GIS*y[15]**2+b_GIS*y[15]+c_GIS+d_GIS*y[9]+e_GIS*(1-y[16]) - d*COUPLING_AGTCCM[0]

                                    ## Definition of the fresh water forcings ##
            # Compution of the F_GIS freshwater flux associated to the melting of GIS # 

            kappa = 11.47 # coefficient based on units conversion to obtain FGIS in [Sv]
            alpha_GIS = kappa*Spot_GIS
            
            F_GIS = -alpha_GIS*dGIS/TAUP[0] # FGIS In [Sv]
            f_gis_list.append(F_GIS)

            # Computation of F_W #
            if f_w_forc==True:
                beta_WAIS = 0.1
                beta_EASB = 0.1
                beta_EAIS = 0.1
                alpha_WAIS = beta_WAIS*kappa*Spot_WAIS
                alpha_EASB = beta_EASB*kappa*Spot_EASB
                alpha_EAIS = beta_EAIS*kappa*Spot_EAIS
                F_W = -(alpha_WAIS*OTE_H[0]/TAUP[2])-(alpha_EASB*OTE_H[1]/TAUP[3])-(alpha_EAIS*OTE_H[2]/TAUP[4])
                f_w_list.append(F_W)
            else :
                F_W = 0 
                f_w_list.append(F_W)

            # Computation of F_E-P # 
            if f_ep_forc==True:
                theta = 0.05
                delta_ep = theta*y[9]
                alpha_ep = 0.3
                F_EP = alpha_ep*delta_ep
                f_ep_list.append(F_EP)
            else:
                F_EP=0
                f_ep_list.append(F_EP)

            # Computation of F_O #
            F_O = F_W+F_EP
            f_o_list.append(F_O)
            
                                            ## AMOC Dynamic ## 
            
            dAMOC = -y[16]**3+a_AMOC*y[16]**2+b_AMOC*y[16]+c_AMOC+d_AMOC*y[9]+e_AMOC*F_GIS+f_AMOC*F_O-d*COUPLING_AGTCCM[1]
            
            GIAM_H = [dGIS,dAMOC] 

            ## Definition of list containing the derivative of the states of each Tipping Elements for each time step ## 
            H_AGTCCM = np.concatenate((GIAM_H,OTE_H),axis=0)

                                ### Definition of the mu timescales regarding the state of the tipping element ###
            MU = H_AGTCCM*0
            for i in range(0,len(H_AGTCCM)) :
                # If the add element state is positive and the tipping element state is lower than 1 
                # then mu is defined as the positive characteristic time scale of the tipping element but 
                # if the tipping element state is already higher than 1 the mu is set to zero. 
                if H_AGTCCM[i]>0 : 
                    if y[15+i]< 1 :
                        MU[i] = 1/TAUP[i]
                    else :
                        MU[i] = 0
                else :
                # If the add element state is negative and the tipping element state is higher than 0.01
                # then mu is defined as the negative characteristic time scale of the tipping element but
                # if the tipping element state is between 0 and 0.01 then mu is set to zero. 
                # if the tipping element is state is negative a negative time scale based on the negative characteristic
                # time scale is set to positively define again the tipping element state. 
                    if y[15+i]> 0.01 :
                        MU[i] = 1/TAUM[i]
                    elif 0 < y[15+i]< 0.01 :
                        MU[i] = 0
                    else : MU[i] = -1/TAUM[i]
                    
            co2_em_tps = cfeedback*CPOT@(MU*H_AGTCCM)*-1
            ch4_em_tps = cfeedback*CH4POT@(MU*H_AGTCCM)*-1
                
        dy = np.zeros(len(y0))
        
        dy[0] = F_volcanism + E + co2_em_tps - F_AU - F_AL - F_weathering + (y[12]-Mch4_pi)/tau_ch4

        dy[1] = F_AL - F_LR - co2_em_landuse - ch4_em_landuse - co2_em_tps

        dy[2] = F_AU - F_UI + F_river
        dy[3] = F_UI - F_ID
        dy[4] = F_ID - F_accumulation*allow_acc 

        dy[5] =  - F_alk_UI + F_alk_river
        dy[6] = F_alk_UI - F_alk_ID
        dy[7] = F_alk_ID - F_alk_accumulation*allow_acc

        dy[8] = (F_accumulation - F_burial)*allow_acc

        dy[9] = (fco2*np.log2(y[0]/MA_pi)+0.036*((y[12]/(mA*carbon_molar_mass)*10**21)**0.5-ch4_pi**0.5)-beta*y[9]+epsi*FMOD@(1-y[15:23])-gamma*(y[9]-y[10]))/(cvol*hu)
        dy[10] = (gamma*(y[9]-y[10])-gamma*(y[10]-y[11]))/(cvol*hi)
        dy[11] = gamma*(y[10]-y[11])/(cvol*hd)

        dy[12] = (Mch4_pi-y[12])/tau_ch4 + ch4_em_landuse + ch4_em_fossil + ch4_em_tps
        dy[13] = -co2_em_landuse - co2_em_tps
        
        dy[14]  = (Sglpot*np.tanh(y[9]/Sglxi)-y[14])/taugl
        
        if AGTCCM==False:
            dy[15:23] = MU*H
        if AGTCCM==True:
            dy[15:23] = MU*H_AGTCCM
        
        if AGTCCM==False:
            dy[23] = 0
            dy[24] = 0
            dy[25] = 0
            dy[26] = 0
        if AGTCCM==True:
            dy[23] = F_GIS
            dy[24] = F_W
            dy[25] = F_EP
            dy[26] = F_O
        
        return dy

    k_list=[k_AU_list,k_UI_list,k_IU_list,k_ID_list,k_DI_list]
    f_list = [f_gis_list,f_w_list,f_ep_list,f_o_list]
    n = int((tend-t0)/dt)+1 ; teval=np.linspace(t0,tend,n)

    if kwargs['int_method'] == 'RK45' : 
        sol=solve_ivp(f, [t0,tend], y0,method= 'RK45',rtol=10**-6,t_eval=teval)

    else : sol=solve_ivp(f, [t0,tend], y0,method= 'LSODA',t_eval=teval, max_step = 10000)
    
    return sol, k_list, f_list

## 2. Weathering

In [168]:
archer_p = np.genfromtxt(data_path+'archer_param.csv', delimiter=',')
x1 = archer_p[:,0]
x2 = archer_p[:,1]
x3 = archer_p[:,2]

CO32_pi = 87
MS_pi = 1750
F_weathering_pi = 0.13


A_acc = np.column_stack((x1-CO32_pi,x2-MS_pi/2,(x1-CO32_pi)*(x2-MS_pi/2)))
b_acc = x3-F_weathering_pi
acc_coef  = np.linalg.lstsq(A_acc, b_acc,rcond=None)[0]

def acc(CO32,M_BL) :
    return F_weathering_pi + (acc_coef[0]*(CO32-CO32_pi)+ acc_coef[1]*(M_BL-MS_pi/2)+ acc_coef[2]*(CO32-CO32_pi)*(M_BL-MS_pi/2))/1

def bur(CO32,M_BL) :
    bur =0.13*M_BL/(MS_pi/2)
    return bur

## 3. Forcing scenarios

### 3.1 SURFER v3.0 Forcings 


In [169]:
################################# other custom forcings #################################

cb = np.genfromtxt(data_path+'carbon_budget.csv', delimiter=',')
np.nan_to_num(cb, copy=False, nan=0.0)

x = np.concatenate(([1750],cb[100:,0]),axis=0)
y = np.concatenate(([0.25],cb[100:,2]),axis=0)
landuse_emissions = interpolate.interp1d(x,y,bounds_error= False, fill_value=0)
fossil_emissions = interpolate.interp1d(cb[:,0],cb[:,1]-cb[:,6],bounds_error= False, fill_value=0)

def emissions(t,toemit,emitted,emitting):
    total = toemit + emitted
    a= -emitting/(toemit*emitted)
    F = (total*toemit*np.exp(a*total*t))/(toemit*np.exp(a*total*t)+emitted)
    E = -a*F*(total-F)
    return E

def forcingCO2(tswitch,toemit,**kwargs):
    
    t=np.linspace(0,2000,2001)
    time = np.linspace(1750,tswitch,tswitch-1750+1)
    hist_em = fossil_emissions(time)
    
    if kwargs['landuse'] == True : 
        hist_em = fossil_emissions(time)+landuse_emissions(time)

    emitting = hist_em[-1]
    emitted = np.sum(hist_em)
    
    if kwargs['mode'] == 'add' : 
        total_toemit = toemit+emitting/2 # emitting/2 is a correction factor to get integrals right
    else : total_toemit = toemit-emitted+emitting/2
    
    x = emissions(t,total_toemit,emitted,emitting) 
    em = np.concatenate([hist_em,x[1:]])       
    t_em = np.concatenate([time-tswitch,t[1:]])

    forcing = interpolate.interp1d(t_em+tswitch-kwargs['origin'],em,bounds_error= False, fill_value=0)

    return forcing

def zero_forcing(teval):
    return interpolate.interp1d(teval,teval*0,bounds_error= False, fill_value=0)

################################# SSP CO2 for this study #################################

co2_ssp_landuse = np.genfromtxt(data_path+'co2_ssp_landuse.csv', delimiter=',')
co2_ssp_total = np.genfromtxt(data_path+'co2_ssp_total.csv', delimiter=',')
ssp_time = co2_ssp_landuse[0,:]

## ssp1-2.6

ssp_1_26_landuse = co2_ssp_landuse[1,:]*12/44*10**-3

yearsbefore = np.arange(1750,1990,1)
yearsafter = np.arange(2101,2151,1)
hist_em =  landuse_emissions(yearsbefore)

long_ssp = ssp_1_26_landuse[-1] -ssp_1_26_landuse[-1]*(yearsafter-2100)/50 
x = np.concatenate((yearsbefore,ssp_time,yearsafter),axis=0)
y = np.concatenate((hist_em,ssp_1_26_landuse,long_ssp),axis=0)
ssp_126_landuse = interpolate.interp1d(x,y,bounds_error= False, fill_value=0)

ssp_1_26_fossil = co2_ssp_total[1,:]*12/44*10**-3-ssp_1_26_landuse

yearsbefore = np.arange(1750,1990,1)
yearsafter = np.arange(2101,2201,1)
hist_em =  fossil_emissions(yearsbefore)

long_ssp = ssp_1_26_fossil[-1] -ssp_1_26_fossil[-1]*(yearsafter-2100)/100 
x = np.concatenate((yearsbefore,ssp_time,yearsafter),axis=0)
y = np.concatenate((hist_em,ssp_1_26_fossil,long_ssp),axis=0)
ssp_126_fossil = interpolate.interp1d(x,y,bounds_error= False, fill_value=0)

time=np.linspace(1750,2500,2500-1750+1)
Y = ssp_126_landuse(time)+ssp_126_fossil(time)
ssp_126_total = interpolate.interp1d(time,Y,bounds_error= False, fill_value=0)

## ssp3-7.0

ssp_3_7_landuse = co2_ssp_landuse[2,:]*12/44*10**-3

yearsbefore = np.arange(1750,1990,1)
yearsafter = np.arange(2101,2151,1)
hist_em =  landuse_emissions(yearsbefore)

long_ssp = ssp_3_7_landuse[-1] -ssp_3_7_landuse[-1]*(yearsafter-2100)/50 
x = np.concatenate((yearsbefore,ssp_time,yearsafter),axis=0)
y = np.concatenate((hist_em,ssp_3_7_landuse,long_ssp),axis=0)
ssp_370_landuse = interpolate.interp1d(x,y,bounds_error= False, fill_value=0)

ssp_3_7_fossil = co2_ssp_total[2,:]*12/44*10**-3-ssp_3_7_landuse

yearsbefore = np.arange(1750,1990,1)
yearsafter = np.arange(2101,2251,1)
hist_em =  fossil_emissions(yearsbefore)

long_ssp = ssp_3_7_fossil[-1] -ssp_3_7_fossil[-1]*(yearsafter-2100)/150 
x = np.concatenate((yearsbefore,ssp_time,yearsafter),axis=0)
y = np.concatenate((hist_em,ssp_3_7_fossil,long_ssp),axis=0)
ssp_370_fossil = interpolate.interp1d(x,y,bounds_error= False, fill_value=0)

time=np.linspace(1750,2500,2500-1750+1)
Y = ssp_370_landuse(time)+ssp_370_fossil(time)
ssp_370_total = interpolate.interp1d(time,Y,bounds_error= False, fill_value=0)


################################# SSP CH4 for this study #################################

ch4_jones_fossil = np.genfromtxt(data_path+'ch4_jones_fossil.csv', delimiter=',')
ch4_jones_landuse = np.genfromtxt(data_path+'ch4_jones_landuse.csv', delimiter=',')


ch4_ssp = np.genfromtxt(data_path+'ch4_ssp.csv', delimiter=',')

ssp_time = ch4_ssp[0,:]

## SSP1-2.6
ch4_ssp126_landuse = np.sum(ch4_ssp[1:5,:],axis=0)*12/16*10**-3
ch4_ssp126_total = ch4_ssp[6,:]*12/16*10**-3
ch4_ssp126_fossil = ch4_ssp126_total -ch4_ssp126_landuse

yearsbefore = np.arange(1750,1990,1)
yearsafter = np.arange(2101,2501,1)

long_ssp = yearsafter*0 + ch4_ssp126_landuse[-1]
x = np.concatenate(([1750],ch4_jones_landuse[:160,4],ssp_time,yearsafter),axis=0)
y = np.concatenate(([0],ch4_jones_landuse[:160,5]/1000*0.75,ch4_ssp126_landuse,long_ssp),axis=0)
ssp_126_landuse_ch4 = interpolate.interp1d(x,y,bounds_error= False, fill_value=0)

yearsbefore = np.arange(1750,1990,1)
yearsafter = np.arange(2101,2251,1)

long_ssp = ch4_ssp126_fossil[-1] -ch4_ssp126_fossil[-1]*(yearsafter-2100)/150
x = np.concatenate(([1750],ch4_jones_fossil[:160,4],ssp_time,yearsafter),axis=0)
y = np.concatenate(([0],ch4_jones_fossil[:160,5]/1000*0.75,ch4_ssp126_fossil,long_ssp),axis=0)
ssp_126_fossil_ch4 = interpolate.interp1d(x,y,bounds_error= False, fill_value=0)

time=np.linspace(1750,2500,2500-1750+1)
Y = ssp_126_landuse_ch4(time)+ssp_126_fossil_ch4(time)
ssp_126_total_ch4 = interpolate.interp1d(time,Y,bounds_error= False, fill_value=0)

## SSP37.0
ch4_ssp370_landuse = np.sum(ch4_ssp[8:12,:],axis=0)*12/16*10**-3
ch4_ssp370_total = ch4_ssp[13,:]*12/16*10**-3
ch4_ssp370_fossil = ch4_ssp370_total -ch4_ssp370_landuse

yearsbefore = np.arange(1750,1990,1)
yearsafter = np.arange(2101,2501,1)

long_ssp = yearsafter*0 + ch4_ssp370_landuse[-1]
x = np.concatenate(([1750],ch4_jones_landuse[:160,4],ssp_time,yearsafter),axis=0)
y = np.concatenate(([0],ch4_jones_landuse[:160,5]/1000*0.75,ch4_ssp370_landuse,long_ssp),axis=0)
ssp_370_landuse_ch4 = interpolate.interp1d(x,y,bounds_error= False, fill_value=0)

yearsbefore = np.arange(1750,1990,1)
yearsafter = np.arange(2101,2251,1)

long_ssp = ch4_ssp370_fossil[-1] -ch4_ssp370_fossil[-1]*(yearsafter-2100)/150
x = np.concatenate(([1750],ch4_jones_fossil[:160,4],ssp_time,yearsafter),axis=0)
y = np.concatenate(([0],ch4_jones_fossil[:160,5]/1000*0.75,ch4_ssp370_fossil,long_ssp),axis=0)
ssp_370_fossil_ch4 = interpolate.interp1d(x,y,bounds_error= False, fill_value=0)

time=np.linspace(1750,2500,2500-1750+1)
Y = ssp_370_landuse_ch4(time)+ssp_370_fossil_ch4(time)
ssp_370_total_ch4 = interpolate.interp1d(time,Y,bounds_error= False, fill_value=0)

### 3.2 Plot of emission scenarios 

In [170]:
### Plot of the VIC scenarios ### 

Nexp = 15
sourceColors = ["#0077BB", "#CC3311",]
targetColors = colInterpolatOr(sourceColors, colorCode = "HEX", nTarget = Nexp)

land_use_bin = False

fig,ax1 = plt.subplots(1,1,figsize=(8,4),sharex= True)

time = np.linspace(0,750,1000)
aa = np.linspace(1000,5000,Nexp)

for i in range(0,Nexp) : 
    
    co2_fossil = forcingCO2(2010,aa[i],mode='total',landuse= land_use_bin, origin=1750) 
    ax1.plot(time+1750,co2_fossil(time),c=targetColors[i],zorder = Nexp-i)

ax1.set_ylabel('PgC/yr')
ax1.set_xlabel(' Years AD')
ax1.set_title('CO2 emission scenarios')

if land_use_bin:
    fig.savefig(save_fig_path_scenarios+"VIC_scenarios_landuse_on.png")
else:
    fig.savefig(save_fig_path_scenarios+"VIC_scenarios_landuse_off.png")

<IPython.core.display.Javascript object>

In [171]:
### Plot of the Methane SSP scenerios ### 


## 4. Helping functions

In [172]:

def k0(T,S) :
     return np.exp(-60.2409+93.4517*(100/T)+23.3585*np.log(T/100)+S*(0.023517-0.023656*(T/100)+0.0047036*(T/100)**2))

def k1(T,S) : 
    mlog10k1 = -62.008+3670.7/T+9.7944*np.log(T)-0.0118*S+0.000116*S**2
    return 10**-mlog10k1
    
def k2(T,S) :
    mlog10k2 = 4.777+1394.7/T-0.0184*S+0.000118*S**2
    return 10**-mlog10k2

def b(M,Q,K1,K2) : 
    return 0.5-Q/(2*M)+(K1*np.sqrt((M-Q)**2-4*K2/K1*Q*(Q-2*M))-4*M*K2)/(2*M*(K1-4*K2))

def h(M,Q,K1,K2) :
    return K1/(2*Q)*(np.sqrt((M-Q)**2-4*K2/K1*Q*(Q-2*M))+M-Q)

def CO32U(M,Q,K1,K2) :
    B = b(M,Q,K1,K2)
    return (Q + M*(B-1))/(WU*carbon_molar_mass)*10**18

def CO32D(M,Q,K1,K2) :
    B = b(M,Q,K1,K2)
    return (Q + M*(B-1))/(WD*carbon_molar_mass)*10**18

def species(sol,T,S,**kwargs):
    
    W =np.array([WU,WI,WD])
    
    if kwargs['ktemp'] == True : 
        ktemp = 1
    else: ktemp = 0
            
    if kwargs['level'] == 'U' :
        i = 0
    elif kwargs['level'] == 'I' : 
        i = 1
    else: i = 2
    
    temp = T + ktemp*sol.y[9+i]
    
    K0 = k0(temp,S)
    K1 = k1(temp,S)
    K2 = k2(temp,S)

    B = b(sol.y[2+i],sol.y[5+i],K1,K2)
    H = h(sol.y[2+i],sol.y[5+i],K1,K2)

    CO3 = (sol.y[5+i] + sol.y[2+i]*(B-1))/(W[i]*carbon_molar_mass)*10**18
    HCO3 = H*CO3/K2
    H2CO3 = H*HCO3/K1
    
    return H,H2CO3,HCO3,CO3


def species_o(sol,T_U,S_U,T_D,S_D):
    
    tempU = T_U 
    
    K0U = k0(tempU,S_U)
    K1U = k1(tempU,S_U)
    K2U = k2(tempU,S_U)
    
    BU = b(sol.y[1],sol.y[4],K1U,K2U)
    HU = h(sol.y[1],sol.y[4],K1U,K2U)
    
    CO3U = (sol.y[4] + sol.y[1]*(BU-1))/(WU*carbon_molar_mass)*10**18
    HCO3U = HU*CO3U/K2U
    H2CO3U = HU*HCO3U/K1U
    
    tempD = T_D + sol.y[8]
    
    K0D = k0(tempD,S_D)
    K1D = k1(tempD,S_D)
    K2D = k2(tempD,S_D)
    
    BD = b(sol.y[2],sol.y[5],K1D,K2D)
    HD = h(sol.y[2],sol.y[5],K1D,K2D)
    
    CO3D = (sol.y[5] + sol.y[2]*(BD-1))/(WD*carbon_molar_mass)*10**18
    HCO3D = HD*CO3D/K2D
    H2CO3D = HD*HCO3D/K1D
    
    return HU,H2CO3U,HCO3U,CO3U,HD,H2CO3D,HCO3D,CO3D

def init_DICU(T_U,S_U,alk_U,pCO2_A):
    
    K0U = k0(T_U,S_U)
    K1U = k1(T_U,S_U)
    K2U = k2(T_U,S_U)
    H2CO3U = K0U*pCO2_A + K0U/(mA*carbon_molar_mass*10**-18*K_AU_pi)*((1-X)*(F_CaCO3_pi+F_CaSiO3_pi)+F_LR)
    HU = (K1U + np.sqrt(K1U**2+8*alk_U*K1U*K2U/H2CO3U))/(2*alk_U/H2CO3U)
    HCO3U = K1U*H2CO3U/HU
    CO3U = K2U*HCO3U/HU
    DIC_U = H2CO3U+HCO3U+CO3U
    
    return DIC_U 

def init_DICD(T_D,S_D,alk_D,CO32_D) : 
    
    K0D = k0(T_D,S_D)
    K1D = k1(T_D,S_D)
    K2D = k2(T_D,S_D)
    HCO3D = alk_D-2*CO32_D
    HD = K2D*HCO3D/CO32_D
    H2CO3D = HD*HCO3D/K1D
    DIC_D = H2CO3D+HCO3D+CO32_D
    
    return DIC_D 

alpha_U = 2.3*10**-4
alpha_I = 1.3*10**-4
alpha_D = 1.3*10**-4

def SLR(sol) :
    S_th = alpha_U*hu*sol.y[9] + alpha_I*hi*sol.y[10] + alpha_D*hd*sol.y[11]
    S_gl = sol.y[14]
    S_GIS = Spot_GIS*(1-sol.y[15])
    S_WAIS = Spot_WAIS*(1-sol.y[16])
    S_EASB = Spot_EASB*(1-sol.y[17])
    S_EAIS = Spot_EAIS*(1-sol.y[18])
    return S_th + S_gl + S_GIS + S_WAIS + S_EASB + S_EAIS

def SLR_th(sol):
    S_th = alpha_U*hu*sol.y[9] + alpha_I*hi*sol.y[10] + alpha_D*hd*sol.y[11]
    return S_th

def SLR_gl(sol):
    S_gl = sol.y[14]
    return S_gl

def SLR_AIS(sol):
    S_WAIS = Spot_WAIS*(1-sol.y[16])
    S_EASB = Spot_EASB*(1-sol.y[17])
    S_EAIS = Spot_EAIS*(1-sol.y[18])
    SLR_AIS = S_WAIS+S_EASB+S_EAIS
    return SLR_AIS

def SLR_GIS(sol):
    S_GIS = Spot_GIS*(1-sol.y[15])
    return S_GIS

def gtc_to_ppm(reserv):
    
    coeff = 2.07
    reserv_ppm = [val_reserv_gtc*(1/coeff) for val_reserv_gtc in reserv]
    return reserv_ppm

def gtc_to_ppm_ch4(reserv_ch4):
    coeff = 2.75
    reserv_ppm_ch4 = [val_reserv_gtc*(1/coeff) for val_reserv_gtc in reserv_ch4]
    return reserv_ppm_ch4

## 5. SURFER parameters for this whole notebook

In [173]:
### Width of the oceanic boxes ### 

hu=150
hi=500
hd=2500

### Concentration parameters ### 

WU=mO*water_molar_mass*hu/(hu+hi+hd)
WI=mO*water_molar_mass*hi/(hu+hi+hd)
WD=mO*water_molar_mass*hd/(hu+hi+hd)

### Other Parameters ### 

V = 0.065
F_CaCO3_pi = 0.065
F_CaSiO3_pi = 0.065
X = 1
F_LR = 0

### Carbon Flux Coefficients ### 

K_AL_pi = 0.025
beta_L=1.7
K_AU_pi = 0.25
K_UI_pi = 0.023*5
K_ID_pi = 0.023*0.35
K_alk_UI = 0.021*5
K_alk_ID = 0.021*0.35

### Other Parameters ### 

T_U = 273.15 + 17.4
S_U = 34.6
alk_U = 2215
pCO2_A = 280
DIC_U = init_DICU(T_U,S_U,alk_U,pCO2_A)

T_D = 273.15 + 1.73
S_D = 34.73
alk_D = 2336
CO32_D = 87
DIC_D = init_DICD(T_D,S_D,alk_D,CO32_D)

T_I = (T_U + T_D)/2
S_I = (S_U + S_D)/2 # not sure if this is valid
alk_I = (alk_U+alk_D)/2 # not sure if this is valid
DIC_I = (DIC_U + DIC_D)/2

### Carbon reservoir value at PI ###

MA_pi = 580.3
ML_pi = 2200
MU_pi = DIC_U*WU*carbon_molar_mass*10**-18
MI_pi = DIC_I*WI*carbon_molar_mass*10**-18
MD_pi = DIC_D*WD*carbon_molar_mass*10**-18
QU_pi = alk_U*WU*carbon_molar_mass*10**-18
QI_pi = alk_I*WI*carbon_molar_mass*10**-18
QD_pi = alk_D*WD*carbon_molar_mass*10**-18

### Other Carbon flux coefficient ### 
K_IU_pi = K_UI_pi*(MU_pi/MI_pi)
K_DI_pi = K_ID_pi*(MI_pi/MD_pi)

# print("-------------------------")
# print("K_UI ={:.3f}".format(K_UI))
# print("K_IU ={:.3f}".format(K_IU))
# print("K_ID ={:.3f}".format(K_ID))
# print("K_DI ={:.3f}".format(K_DI))
# print("-------------------------")

ch4_pi = 720
Mch4_pi = ch4_pi*mA*carbon_molar_mass*10**-21

---
# Tipping points Model
---

## 1. Calibration Module

### A. Default Calibration Module of SURFER v3.0
To be consistent the notations for coefficient are kept from the "Tipping Module Calibration ParamB.ipynb"

In [174]:
def xm_today(op,om,xp) :
    G = (op+om+2*(op*om)**0.5)/(op-om)
    return (-2+xp*(1+G**(1/3)+G**(-1/3)))/(-1+G**(1/3)+G**(-1/3))

def a2(op,om,xp,xm):
    return 3/2*(xp+xm)

def a1(op,om,xp,xm):
    return -3*xp*xm

def c1(op,om,xp,xm):
    return -(xp-xm)**3/(2*(op-om))

def c0(op,om,xp,xm):
    return (op*xm**2*(xm-3*xp)-om*xp**2*(xp-3*xm))/(2*(om-op))

def params(op,om,xp) : 
    xm = xm_today(op,om,xp)
    a_2 = a2(op,om,xp,xm)
    a_1 = a1(op,om,xp,xm)
    c_1 = c1(op,om,xp,xm)
    c_0 = c0(op,om,xp,xm)
    return a_2,a_1,c_1,c_0


### B. AMOC-GIS Tipping Cascade Calibration Module (AGTCCM)  

To remain coherent with the already used notations in SURFER v3.0 and preliminary version the notations of AGTCCM from "Tipping Module Calibration ParamB.ipynb" had to be adapted

Notations from AGTCCM $\to$ Notations from SURFER

$$a_{i} \to a_2$$
$$b_{i} \to a_1$$
$$c_{i} \to c_1$$
$$d_{i} \to c_0$$

In AGTCCM's notations, the indices $i$ stands for the tipping element considered,

$$1 \to AMOC $$
$$2 \to GIS $$

Whereas in SURFER's notations the tipping element is directly written in the variable's name.

For the "General Calibration Functions" I keep the SURFER's notations but in the code associated to the "Specific AMOC-GIS Calibration Functions" I keep the AGTCCM notation convention. 

In [175]:
########## Definition of the functions computing the coefficient's values. ##########

##### General Calibration Functions #####

## General functions shared by all tipping elements ## 
def a2(op,om,xp,xm):
    """Definition of the a_2 coefficient cfr. Eq.(?)"""
    return (3/2)*(xp+xm)
def a1(op,om,xp,xm):
    """Definition of the a_1 coefficient cfr. Eq.(?)"""
    return -3*xp*xm
def c1(op,om,xp,xm):
    """Definition of the c_1 coefficient cfr. Eq.(?)"""
    return -(xp-xm)**3/(2*(op-om))
def xm_today(op,om,xp) :
    """Definition of the constrain that the state of the system is equal to 1, its maximal value when the forcing is equal to zero at pre-industrial times"""
    G = (op+om+2*(op*om)**0.5)/(op-om)
    return (-2+xp*(1+G**(1/3)+G**(-1/3)))/(-1+G**(1/3)+G**(-1/3))

## General functions shared by all tipping elements except GIS and AMOC in the AGTCCM ## 

def c0(op,om,xp,xm):
    """Definition of the c_0 coefficient cfr. Eq.(?)"""
    return (op*xm**2*(xm-3*xp)-om*xp**2*(xp-3*xm))/(2*(om-op))

##### Specific AMOC-GIS Calibration Functions #####

## Functions specific to the AMOC's system in AGTCCM ## 

def e12(xp,xm,op,om):
    """Definition of the e_12 coefficient cfr. Eq.(?)"""
    return -(xp-xm)**3/(2*(op-om))
def f(xp,xm,op,om):
    """Definition of the f coefficient cfr. Eq.(?)"""
    return -(xp-xm)**3/(2*(op-om))

def cAMOC_A(xp,xm,op,om):
    """Definition of the c_1^A coefficient cfr. Eq.(?)"""
    return (op*xm**2*(xm-3*xp)-om*xp**2*(xp-3*xm))/(2*(om-op))
def cAMOC_B(xp,xm,op,om):
    """Definition of the c_1^B coefficient cfr. Eq.(?)"""
    return (op*xm**2*(xm-3*xp)-om*xp**2*(xp-3*xm))/(2*(om-op))
def cAMOC_C(xp,xm,op,om):
    """Definition of the c_1^C coefficient cfr. Eq.(?)"""
    return (op*xm**2*(xm-3*xp)-om*xp**2*(xp-3*xm))/(2*(om-op))

def c1_from_A(xp,xm,op1,om1, e12, FGIS_A, f, FO_A):
    """Definition of the c_1 coefficient from experience A cfr. Eq.(?)"""
    c_A = (op1*xm**2*(xm-3*xp)-om1*xp**2*(xp-3*xm))/(2*(om1-op1))
    return (c_A-(e12*FGIS_A)-(f*FO_A))
def c1_from_B(xp,xm,op2,om2, d1, TAMOC_B, f, FO_B):
    """Definition of the c_1 coefficient from experience B cfr. Eq.(?)"""
    c_B = (op2*xm**2*(xm-3*xp)-om2*xp**2*(xp-3*xm))/(2*(om2-op2))
    return c_B-(d1*TAMOC_B)-(f*FO_B)
def c1_from_C(xp,xm,op3,om3, d1, TAMOC_C, e12, FGIS_C):
    """Definition of the c_1 coefficient from experience C cfr. Eq.(?)"""
    c_C = (op3*xm**2*(xm-3*xp)-om3*xp**2*(xp-3*xm))/(2*(om3-op3))
    return c_C-(d1*TAMOC_C)-(e12*FGIS_C)

## Functions specific to the GIS's system in AGTCCM ## 

def e21(xp,xm,op,om):
    """Definition of the e_21 coefficient cfr. Eq.(?)"""
    return -(xp-xm)**3/(2*(om-op))
def c2_from_C(xp,xm,op1,om1,op2,om2,Psi_C):
    """Definition of the c_2 coefficient from experience C cfr. Eq.(?)"""
    c_C = (op1*xm**2*(xm-3*xp)-om1*xp**2*(xp-3*xm))/(2*(om1-op1))
    e_21 = e21(xp,xm,op2,om2)
    return (c_C - (e_21*(1-Psi_C)))
def c2_from_D(xp,xm,op2,om2,op1,om1,TGIS_D):
    """Definition of the c_2 coefficient from experience D cfr. Eq.(?)"""
    e_21 = e21(xp,xm,op2,om2)
    a_2 = a2(op1,om1,xp,xm)
    b_2 = a1(op1,om1,xp,xm)
    c_D = ((xp**3) - a_2*(xp**2) - b_2*xp - e_21*(1-op2))
    d_2 = c1(op1,om1,xp,xm)
    return (c_D-(d_2*TGIS_D))

### Definition of the calibration module ###

## AMOC ## 
def calib_module_AMOC_ParamB(Psip,Psim, Tp, Tm, FGISp, FGISm, FOp, FOm, FGIS_A, FO_A, TAMOC_B, FO_B, TAMOC_C, FGIS_C):
    """Calibration module for the AMOC with the parametrisation B for the fresh water flux."""
    #Psim = xm_today(Tp, Tm, Psip)
    a_1 = a2(Tp,Tm,Psip,Psim)
    b_1 = a1(Tp,Tm,Psip,Psim)
    d_1 = c1(Tp,Tm,Psip,Psim)
    e_12 = e12(Psip,Psim,FGISp,FGISm)
    f_1 = f(Psip,Psim,FOp,FOm)
    c1_f_A = c1_from_A(Psip, Psim, Tp, Tm, e_12, FGIS_A, f_1, FO_A)
    c1_f_B = c1_from_B(Psip, Psim, FGISp, FGISm, d_1, TAMOC_B, f_1, FO_B)
    c1_f_C = c1_from_C(Psip, Psim, FOp, FOm, d_1, TAMOC_C, e_12, FGIS_C)
    return [a_1, b_1, d_1, e_12, f_1, c1_f_A, c1_f_B, c1_f_C]

## GIS ## 
def calib_module_GIS(Vp, Vm, Tp, Tm, Psip, Psim, Psi_C, TGIS_D):
    """Calibration module for the GIS"""
    Vm = xm_today(Tp,Tm,Vp)
    a_2 = a2(Tp,Tm,Vp,Vm)
    b_2 = a1(Tp,Tm,Vp,Vm)
    d_2 = c1(Tp,Tm,Vp,Vm)
    e_21 = e21(Vp, Vm, Psip, Psim)
    c2_f_C = c2_from_C(Vp, Vm, Tp, Tm, Psip, Psim, Psi_C)
    c2_f_D = c2_from_D(Vp, Vm, Psip, Psim, Tp, Tm, TGIS_D)
    return [a_2, b_2, d_2, e_21, c2_f_C, c2_f_D]


---
# Simulations - CONF_A 
---
CONF_A  = Victor Couplet scenarios (originating from SURFER v3.0) on SURFER v4.0 with configuration with : 
- Activated AMOC-GIS Tipping Cascade Calibration Module 
- Normal case coupling in the Tipping Point Model 
- Activated interactions between the tipping elements 
- Activated AMOC Collapse Impacts on carbon cycle 

## Setting Up

###  AGTCCM  

In [176]:
AGTCCM = True

if AGTCCM == False:
    adv_param_amoc_vic = False

###  cGenie Calibration for AMOC

In [177]:
if AGTCCM == True:
    cGenie_calib = True 
if AGTCCM == False:
    cGenie_calib = False

###  Methane cycle

In [178]:
meth_te_emi = True 
meth_fossil_emi = False 

### Tipping Element Coupling and Feedback

In [179]:
### Couplings between tipping points ###
if AGTCCM == False : 

    S_moy = np.zeros((8,8))

    S_moy[0,1] = -11/2 # AMOC -> GIS
    S_moy[0,2] = 1.5 # WAIS -> GIS
    S_moy[0,3] = 1.5 # EASB -> GIS
    
        
    S_moy[2,0] = 11/2 # GIS -> WAIS
    S_moy[2,1] = 1.25 # AMOC -> WAIS

    S_moy[3,0] = 11/2 # GIS -> EASB
    S_moy[3,1] = 1.25 # AMOC -> EASB

    S_moy[5,1] = -1.5 # AMOC -> AWSI
    # S_moy[4,5] = 0 # AMOC -> AWSI

    S_moy[1,5] = 1.5 # AWSI -> AMOC
    # S_moy[5,4] = 0 # AWSI -> AMOC

    S_moy[6,1] = 0.5 # AMOC -> AMAZ

    S_moy = S_moy/5

    ## Output ## 
    # print("-----------------------")
    # print("S_moy_v2 =")
    # print(S_moy)
    # print("-----------------------")

    # Additionnal coupling coefficient for AMOC # 
    c_10 = (11/2)/5*2900 # dGIS -> AMOC
    c_12 = 0 # dWAIS -> AMOC
    c_13 = 0 # dEASB -> AMOC

if AGTCCM == True : 
    
    S_moy_AGTCCM = np.zeros((8,8))

    S_moy_AGTCCM[0,1] = 0 # AMOC -> GIS # we will encode the coupling effect of AMOC on GIS by the parameterization 
                          # in AGTCC 
    S_moy_AGTCCM[0,2] = 1.5 # WAIS -> GIS
    S_moy_AGTCCM[0,3] = 1.5 # EASB -> GIS
    
        
    S_moy_AGTCCM[2,0] = 11/2 # GIS -> WAIS
    S_moy_AGTCCM[2,1] = 1.25 # AMOC -> WAIS

    S_moy_AGTCCM[3,0] = 11/2 # GIS -> EASB
    S_moy_AGTCCM[3,1] = 1.25 # AMOC -> EASB

    S_moy_AGTCCM[5,1] = -1.5 # AMOC -> AWSI
    # S_moy[4,5] = 0 # AMOC -> AWSI

    S_moy_AGTCCM[1,5] = 1.5 # AWSI -> AMOC
    # S_moy[5,4] = 0 # AWSI -> AMOC

    S_moy_AGTCCM[6,1] = 0.5 # AMOC -> AMAZ

    S_moy_AGTCCM = S_moy_AGTCCM/5

    S_moy_AGTCCM_OTE = np.delete(S_moy_AGTCCM, (0,1), axis=0)

    GIS_S_moy = S_moy_AGTCCM[0,:]
    AMOC_S_moy = S_moy_AGTCCM[1,:]

    S_moy_AGTCCM_GIAM = np.zeros((2,8))
    S_moy_AGTCCM_GIAM[0,:] = GIS_S_moy
    S_moy_AGTCCM_GIAM[1,:] = AMOC_S_moy

    ## Output ##
    # print("S_moy_AGTCCM v2 = ")
    # print(S_moy_AGTCCM)
    # print("-----------------------")
    # print("S_moy_AGTCCM_OTE")
    # print(S_moy_AGTCCM_OTE)
    # print("-----------------------")
    # print("S_moy_AGTCCM_GIAM")
    # print(S_moy_AGTCCM_GIAM)
    # print("-----------------------")

### Feedback on radiative forcing ###

fmod_GIS = 0.43
fmod_AMOC = 0.00
fmod_WAIS = 0.18
fmod_EASB = 0.18
fmod_EAIS = 0.60
fmod_AWSI = 0.60
fmod_AMAZ = 0.00
fmod_PERM = 0.00

### Feedback on emmissions ###

cpot_GIS = 0
cpot_AMOC = 0
cpot_WAIS = 0
cpot_EASB = 0
cpot_EAIS = 0
cpot_AWSI = 0
cpot_AMAZ = 75
cpot_PERM = 500 *0.8

ch4pot_GIS = 0
ch4pot_AMOC = 0
ch4pot_WAIS = 0
ch4pot_EASB = 0
ch4pot_EAIS = 0
ch4pot_AWSI = 0
if meth_te_emi == True:
    ch4pot_AMAZ = 75
    ch4pot_PERM = 500 *0.2
else:
    ch4pot_AMAZ = 0
    ch4pot_PERM = 0

### Normal Case Tipping Element States

In [180]:

##### Critical Values for Tipping Elements and coefficient computation #####

### 0 - GIS ###

## Retrieval of the calibration coefficients based on the AGTCCM or not ##
if AGTCCM==True:
    Tp_GIS = 1.52
    Tm_GIS = 0.3
    Vp_GIS = 0.77
    taup_GIS = 2000
    taum_GIS = 2000
    # Definition of the other critical parameters # 
    Vm_GIS = xm_today(Tp_GIS, Tm_GIS, Vp_GIS) # Lower critical value of the GIS volume [Adim]
    Psip_GIS = 0.7 # Higher critical AMOC's intensity value for the GIS (Non defined in Couplet (2023))
    Psim_GIS = 0.1 # Lower critical AMOC's intensity value for the GIS (Non defined in Couplet (2023))
    # Definition of the sensibility experiment parameters #
    Psi_C = 0.9 # Constant value of the AMOC's intensity for the sensibility experiment C [Adim]
    TGIS_D = 1.5 # Constant value of the temperature for the sensibility experiment D [°C]
    # Computation of the calibration coefficients based on AGTCCM # 
    GIS_Module_calib_coeff = calib_module_GIS(Vp_GIS,Vm_GIS,Tp_GIS,Tm_GIS,Psip_GIS,Psim_GIS,Psi_C,TGIS_D)
    a_GIS = GIS_Module_calib_coeff[0]
    b_GIS = GIS_Module_calib_coeff[1]
    c_GIS = GIS_Module_calib_coeff[4]
    d_GIS = GIS_Module_calib_coeff[2]
    e_GIS = GIS_Module_calib_coeff[3]
    print("---------------------------------")
    print("a_GIS = {:.3f}".format(a_GIS))
    print("b_GIS = {:.3f}".format(b_GIS))
    print("c_GIS = {:.3f}".format(c_GIS))
    print("d_GIS = {:.3f}".format(d_GIS))
    print("e_GIS = {:.3f}".format(e_GIS))
    print("---------------------------------")
if AGTCCM==False:
    Tp_GIS = 1.52
    Tm_GIS = 0.3
    Vp_GIS = 0.77
    taup_GIS = 2000
    taum_GIS = 2000
    a2_GIS,a1_GIS,c1_GIS,c0_GIS = params(Tp_GIS,Tm_GIS,Vp_GIS)
    print("---------------------------------")
    print("a2_GIS = {:.3f}".format(a2_GIS))
    print("a1_GIS = {:.3f}".format(a1_GIS))
    print("c1_GIS = {:.3f}".format(c1_GIS))
    print("c0_GIS = {:.3f}".format(c0_GIS))
    print("---------------------------------")

### 1 - WAIS ###

Tp_WAIS = 1.5
Tm_WAIS = 0.3
Vp_WAIS = 0.75
taup_WAIS = 2000
taum_WAIS = 2000
a2_WAIS,a1_WAIS,c1_WAIS,c0_WAIS = params(Tp_WAIS,Tm_WAIS,Vp_WAIS)

### 2 - EASB ###

Tp_EASB = 3
Tm_EASB = 0.3
Vp_EASB = 0.75
taup_EASB = 2000
taum_EASB = 2000
a2_EASB,a1_EASB,c1_EASB,c0_EASB = params(Tp_EASB,Tm_EASB,Vp_EASB)

### 3 - EAIS ###

Tp_EAIS = 7.5
Tm_EAIS = 4.0
Vp_EAIS = 0.44
taup_EAIS = 10000
taum_EAIS = 10000
a2_EAIS,a1_EAIS,c1_EAIS,c0_EAIS = params(Tp_EAIS,Tm_EAIS,Vp_EAIS)

### 4 - AWSI ###

Tp_AWSI = 6.3
Tm_AWSI = 2.5
Vp_AWSI = 0.7
taup_AWSI = 20
taum_AWSI = 20
a2_AWSI,a1_AWSI,c1_AWSI,c0_AWSI = params(Tp_AWSI,Tm_AWSI,Vp_AWSI)

### 5 - AMOC ###

## Retrieval of the calibration coefficients based on the AGTCCM or not ##
if AGTCCM==True:

    if cGenie_calib == False : 
        Tp_AMOC = 4
        Tm_AMOC = 0
        Psip_AMOC = 0.77
        taup_AMOC = 50
        taum_AMOC = 50
        # Definition of the other critical parameters # 
        Psim_AMOC = xm_today(Tp_AMOC, Tm_AMOC, Psip_AMOC) # Lower critical value of the AMOC Intensity [Adim]
        FGISp_AMOC = 0.5 # Higher critical fresh water forcing value from the GIS for the AMOC (Non defined in Couplet (2023))
        FGISm_AMOC = 0.05 # Lower critical fresh water forcing value from the GIS for the AMOC (Non defined in Couplet (2023))
        FOp_AMOC = 0.8 # Higher critical fresh water forcing value from the other components for the AMOC (Non defined in Couplet (2023))
        FOm_AMOC = 0.08 # Lower critical fresh water forcing value from the other components for the AMOC (Non defined in Couplet (2023))
        # Definition of the sensibility experiment parameters #
        FGIS_A = 0.8 # Constant value of the fresh water flux from the GIS for the sensibility experiment A [Sv] 
        FO_A = 0.3 # Constant value of the fresh water flux from the Other components for the sensibility experiment A [Sv]
        TAMOC_B = 2 # Constant value of the temperature for the sensibility experiment B [°C]
        FO_B = 0.8 # Constant value of the fresh water flux from the Other components for the sensibility experiment B [Sv]
        TAMOC_C = 2 # Constant value of the temperature for the sensibility experiment C [°C]
        FGIS_C = 0.5 # Constant value of the fresh water flux from the GIS for the sensibility experiment C [Sv] 
        # Computation of the calibration coefficients based on AGTCCM # 
        AMOC_Module_calib_coeff= calib_module_AMOC_ParamB(Psip_AMOC,Psim_AMOC,Tp_AMOC,Tm_AMOC,FGISp_AMOC,FGISm_AMOC,FOp_AMOC,FOm_AMOC,FGIS_A,FO_A,TAMOC_B,FO_B,TAMOC_C,FGIS_C)
        a_AMOC = AMOC_Module_calib_coeff[0]
        b_AMOC = AMOC_Module_calib_coeff[1]
        c_AMOC = AMOC_Module_calib_coeff[5]
        d_AMOC = AMOC_Module_calib_coeff[2]
        e_AMOC = AMOC_Module_calib_coeff[3]
        f_AMOC = AMOC_Module_calib_coeff[4]
        print("---------------------------------")
        print("a_AMOC = {:.3f}".format(a_AMOC))
        print("b_AMOC = {:.3f}".format(b_AMOC))
        print("c_AMOC = {:.3f}".format(c_AMOC))
        print("d_AMOC = {:.3f}".format(d_AMOC))
        print("e_AMOC = {:.3f}".format(e_AMOC))
        print("f_AMOC = {:.3f}".format(f_AMOC))
        print("---------------------------------")

    if cGenie_calib == True:
        # Definition of the other critical parameters # 
        Psip_AMOC = 0.6
        Psim_AMOC = 0.022
        Tp_AMOC = 5.5
        Tm_AMOC = 1.27
        FGISp_AMOC = 0.03
        FGISm_AMOC = -0.1
        FOp_AMOC = 0.065
        FOm_AMOC = -0.02
        taup_AMOC = 50
        taum_AMOC = 50
        # Definition of the sensibility experiment parameters #
        F_GIS_cst_A = 0
        F_O_cst_A = 0
        T_forcing_cst_B = 0
        F_O_cst_B = 0
        T_forcing_cst_C = 0
        F_GIS_cst_C = 0
        # Computation of the calibration coefficients based on AGTCCM # 
        AMOC_Module_calib_coeff = calib_module_AMOC_ParamB(Psip_AMOC,Psim_AMOC,Tp_AMOC,Tm_AMOC,FGISp_AMOC,FGISm_AMOC,FOp_AMOC,FOm_AMOC,F_GIS_cst_A, F_O_cst_A, T_forcing_cst_B, F_O_cst_B, T_forcing_cst_C, F_GIS_cst_C)
        a_AMOC = AMOC_Module_calib_coeff[0]
        b_AMOC = AMOC_Module_calib_coeff[1]
        c_AMOC = AMOC_Module_calib_coeff[5]
        c_bis1 = AMOC_Module_calib_coeff[6]
        c_bis2 = AMOC_Module_calib_coeff[7]
        d_AMOC = AMOC_Module_calib_coeff[2]
        e_AMOC = AMOC_Module_calib_coeff[3]
        f_AMOC = AMOC_Module_calib_coeff[4]
        print("---------------------------------")
        print("a_AMOC = {:.3f}".format(a_AMOC))
        print("b_AMOC = {:.3f}".format(b_AMOC))
        print("c_AMOC = {:.3f}".format(c_AMOC))
        print("d_AMOC = {:.3f}".format(d_AMOC))
        print("e_AMOC = {:.3f}".format(e_AMOC))
        print("f_AMOC = {:.3f}".format(f_AMOC))
        print("---------------------------------")

if AGTCCM==False:
    Tp_AMOC = 4
    Tm_AMOC = 0
    Vp_AMOC = 0.77
    taup_AMOC = 50
    taum_AMOC = 50
    a2_AMOC,a1_AMOC,c1_AMOC,c0_AMOC = params(Tp_AMOC,Tm_AMOC,Vp_AMOC)
    print("---------------------------------")
    print("a2_AMOC = {:.3f}".format(a2_AMOC))
    print("a1_AMOC = {:.3f}".format(a1_AMOC))
    print("c1_AMOC = {:.3f}".format(c1_AMOC))
    print("c0_AMOC = {:.3f}".format(c0_AMOC))
    print("---------------------------------")


### 6 - AMAZ ###

Tp_AMAZ = 3.5
Tm_AMAZ = 0
Vp_AMAZ = 0.77
taup_AMAZ = 100
taum_AMAZ = 100
a2_AMAZ,a1_AMAZ,c1_AMAZ,c0_AMAZ = params(Tp_AMAZ,Tm_AMAZ,Vp_AMAZ)

### 7 - PERM ###

Tp_PERM = 4
Tm_PERM = 0
Vp_PERM = 0.75
taup_PERM = 50
taum_PERM = 50
a2_PERM,a1_PERM,c1_PERM,c0_PERM = params(Tp_PERM,Tm_PERM,Vp_PERM)

### SLR ###

Spot_GIS = 7.42 # value updated from Morlighem et al.(2017)
Spot_WAIS = 5
Spot_EASB = 9
Spot_EAIS = 45

##### Vectors of the coefficients and parameters #####

if AGTCCM==False:
    A2 = np.array([a2_GIS,a2_AMOC,a2_WAIS,a2_EASB,a2_EAIS,a2_AWSI,a2_AMAZ,a2_PERM])
    A1 = np.array([a1_GIS,a1_AMOC,a1_WAIS,a1_EASB,a1_EAIS,a1_AWSI,a1_AMAZ,a1_PERM])
    C1 = np.array([c1_GIS,c1_AMOC,c1_WAIS,c1_EASB,c1_EAIS,c1_AWSI,c1_AMAZ,c1_PERM])
    C0 = np.array([c0_GIS,c0_AMOC,c0_WAIS,c0_EASB,c0_EAIS,c0_AWSI,c0_AMAZ,c0_PERM])
if AGTCCM==True:
    A2_AGTCCM = np.array([a2_WAIS,a2_EASB,a2_EAIS,a2_AWSI,a2_AMAZ,a2_PERM])
    A1_AGTCCM = np.array([a1_WAIS,a1_EASB,a1_EAIS,a1_AWSI,a1_AMAZ,a1_PERM])
    C1_AGTCCM = np.array([c1_WAIS,c1_EASB,c1_EAIS,c1_AWSI,c1_AMAZ,c1_PERM])
    C0_AGTCCM = np.array([c0_WAIS,c0_EASB,c0_EAIS,c0_AWSI,c0_AMAZ,c0_PERM])

TAUP = np.array([taup_GIS,taup_AMOC,taup_WAIS,taup_EASB,taup_EAIS,taup_AWSI,taup_AMAZ,taup_PERM])
TAUM = np.array([taum_GIS,taum_AMOC,taum_WAIS,taum_EASB,taum_EAIS,taum_AWSI,taum_AMAZ,taum_PERM])

FMOD = np.array([fmod_GIS,fmod_AMOC,fmod_WAIS,fmod_EASB,fmod_EAIS,fmod_AWSI,fmod_AMAZ,fmod_PERM])
CPOT = np.array([cpot_GIS,cpot_AMOC,cpot_WAIS,cpot_EASB,cpot_EAIS,cpot_AWSI,cpot_AMAZ,cpot_PERM])
CH4POT = np.array([ch4pot_GIS,ch4pot_AMOC,ch4pot_WAIS,ch4pot_EASB,ch4pot_EAIS,ch4pot_AWSI,ch4pot_AMAZ,ch4pot_PERM])

---------------------------------
a_GIS = 1.684
b_GIS = -0.815
c_GIS = 0.125
d_GIS = -0.030
e_GIS = 0.061
---------------------------------
---------------------------------
a_AMOC = 0.933
b_AMOC = -0.040
c_AMOC = 0.029
d_AMOC = -0.023
e_AMOC = -0.743
f_AMOC = -1.136
---------------------------------


### Coupling between Tipping Elements

In [181]:
cfeedback = 1
d= 0.5 # Coupling between tipping eleements # 
epsi = 1

### Freshwater forcing

In [182]:
if AGTCCM == True:
    f_w_forc = True
    f_ep_forc = False
if AGTCCM == False:
    f_w_forc = False
    f_ep_forc = False

### AMOC Collapse Impacts on Carbon Cycle

In [183]:
### Activation of the AMOC impacts on carbon cycle model ### 
if AGTCCM==True:
    amoc_impact_carb = True

    amoc_impact_carb_au = False # Was used for test but has to remain on False due to physical reasons

    impact_k_ui = True
    impact_k_iu = True

    amoc_impact_carb_id = True
    impact_k_id = True
    impact_k_di = True

    ## Calibration coefficients for the impacts on carbon cycle model ##
    c_Sto = -0.3
    c_mean_o = 0.01
    c_mean_a = 0.3

    c_AU = c_Sto*c_mean_a

    c_UI = c_Sto*c_mean_o
    c_IU = c_Sto*c_mean_o

    c_ID = c_Sto*c_mean_o
    c_DI = c_Sto*c_mean_o

if AGTCCM ==False:
    amoc_impact_carb = False

    amoc_impact_carb_au = False # Was used for test but has to remain on False due to physical reasons

    impact_k_ui = False
    impact_k_iu = False

    amoc_impact_carb_id = False
    impact_k_id = False
    impact_k_di = False

## Simulations

In [184]:
                                        ##### Simulations Parameters #####
### Choosing Number of Experiences and Scenarios ###
Nexp = 15
### Choosing Time of Simulations ###
sim_2300=False
sim_2500=False
sim_5000=True
sim_10000=False
sim_100000=False
sim_500000=False

if sim_2300==True:
    tend = 550
    save_fig_path2='/SIM 2300/'
elif sim_2500==True: 
    tend = 750
    save_fig_path2='/SIM 2500/'
elif sim_5000==True:
    tend = 3250
    save_fig_path2='/SIM 5000/'
elif sim_10000==True:
    tend = 8250
    save_fig_path2='/SIM 10 000/'
elif sim_100000==True:
    tend = 98250
    save_fig_path2='/SIM 100 000/'
elif sim_500000==True:
    tend=500000
    save_fig_path2='/SIM 500 000/'
t0=0 # Initial time 
dt=1 # Time step 
n = int((tend-t0)/dt)+1 # Number of time steps 
teval=np.linspace(t0,tend,n) # List of discrete time at which solutions are stored

                                            ##### Initial State ######

if cGenie_calib==True:
    psi_0 = 0.9284
else:
    psi_0 = 1
y0 = [MA_pi,ML_pi,MU_pi,MI_pi,MD_pi,QU_pi,QI_pi,QD_pi,MS_pi,0,0,0,Mch4_pi,ML_pi,0,1,psi_0,1,1,1,1,1,1,0,0,0,0] # Initial Vector State

co2_landuse = zero_forcing(teval) # Land use CO2 parameterisation 
ch4_landuse = zero_forcing(teval) # Land use CH2 parameterisation 
if meth_fossil_emi==False:
    ch4_fossil = zero_forcing(teval) # Fossil CH4 parameterisation


                                        ##### Solutions Computations #####

tot_emit_scen = np.linspace(1000,5000,Nexp) # List with the different total CO2 emissions in PgC that will be used to create the emission scenarios
solutions_CONF_A_VIC = [] # Array to store the solutions 
solutions_CONF_A_VIC_carb_impact_param = [] # Array to store the carbon flux coefficients
solutions_CONF_A_VIC_f = [] # Array to store the solutions related to the freshwater fluxes

sourceColors = ["#0077BB", "#CC3311",] # Color interpolator 
targetColors = colInterpolatOr(sourceColors, colorCode = "HEX", nTarget = Nexp)

for i in range(0,Nexp) : 
    
    co2_fossil = forcingCO2(2010,tot_emit_scen[i],mode='total',landuse= True, origin=1750) 
    sol, sol_carb, sol_f = SURFER_T(co2_fossil,co2_landuse,ch4_fossil,ch4_landuse,y0,t0,tend,dt,ktemp = True,sediments = True ,weathering = True,atmtoland_flux='old', int_method = 'LSODA')
    solutions_CONF_A_VIC.append(sol)
    solutions_CONF_A_VIC_carb_impact_param.append(sol_carb)
    solutions_CONF_A_VIC_f.append(sol_f)

    print("Scenario {} with : {:.2f} PgC of total emission done".format(i+1,tot_emit_scen[i])) 

Scenario 1 with : 1000.00 PgC of total emission done
Scenario 2 with : 1285.71 PgC of total emission done
Scenario 3 with : 1571.43 PgC of total emission done
Scenario 4 with : 1857.14 PgC of total emission done
Scenario 5 with : 2142.86 PgC of total emission done
Scenario 6 with : 2428.57 PgC of total emission done
Scenario 7 with : 2714.29 PgC of total emission done
Scenario 8 with : 3000.00 PgC of total emission done
Scenario 9 with : 3285.71 PgC of total emission done


Scenario 10 with : 3571.43 PgC of total emission done
Scenario 11 with : 3857.14 PgC of total emission done
Scenario 12 with : 4142.86 PgC of total emission done
Scenario 13 with : 4428.57 PgC of total emission done
Scenario 14 with : 4714.29 PgC of total emission done
Scenario 15 with : 5000.00 PgC of total emission done


## Results 

### 1. Bifurcation Diagram 

In [185]:
colors = ['C0','C1','C2','C3','C4','C5','C6','C7']

fig, ax = plt.subplots(1,1,figsize=(8,5))

if AGTCCM == False:

    labels = ['GIS','AMOC','WAIS','EASB','EAIS','AWSI','AMAZ','PERM']

    for i in range(0,8):
        
        x = np.linspace(-0.8,1.2,1000)
        y = -1*(-x**3+A2[i]*x**2+A1[i]*x+C0[i])/C1[i]

        ax.plot(y[x>0],x[x>0],c=colors[i],label = labels[i])
        ax.plot([y[x>0][0],17],[0,0],c=colors[i])
        
    ax.set_xlim((-0.5,10))
    ax.legend(fontsize=12)

    ax.set_xlabel('T (°C)')
    ax.set_ylabel('Tipping element state')

if AGTCCM == True:

    labels = ['GIS','AMOC','WAIS','EASB','EAIS','AWSI','AMAZ','PERM']

    x = np.linspace(-0.8,1.2,1000)

    ## Plot of the bifurcation diagram for GIS and AMOC ## 

    yGIS = (x**3-a_GIS*x**2-b_GIS*x-c_GIS-e_GIS*(1-Psi_C))/d_GIS
    yAMOC = (x**3-a_AMOC*x**2-b_AMOC*x-c_AMOC-e_AMOC*F_GIS_cst_A-f_AMOC*F_O_cst_A)/d_AMOC

    ax.plot(yGIS[x>0],x[x>0],c=colors[0],label = labels[0])
    ax.plot(yAMOC[x>0],x[x>0],c=colors[1],label = labels[1])

    ax.plot([yGIS[x>0][0],17],[0,0],c=colors[0])
    ax.plot([yAMOC[x>0][0],17],[0,0],c=colors[1])

    ## Plot of the bifurcation diagram for all the others Tipping Elements ##  
    for i in range(0,6):
        x = np.linspace(-0.8,1.2,1000)
        y = -1*(-x**3+A2_AGTCCM[i]*x**2+A1_AGTCCM[i]*x+C0_AGTCCM[i])/C1_AGTCCM[i]

        ax.plot(y[x>0],x[x>0],c=colors[i+2],label = labels[i+2])
        ax.plot([y[x>0][0],17],[0,0],c=colors[i+2])
        
    ax.set_xlim((-0.5,10))
    ax.legend(fontsize=12)

    ax.set_xlabel('T [°C]',fontsize=14)
    ax.set_ylabel('State of the Tipping Elements [Adim]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.title("Bifurcation Diagrams",fontsize=20)

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_bifdiag.png",dpi=300)

<IPython.core.display.Javascript object>

### 2. Temperature

#### 2.1 $\delta T_U$

In [186]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        ax.plot(sol.t+1750,sol.y[9],c=targetColors[i],zorder = Nexp-i)

    plt.title('Temperature Anomaly '+ r'$\delta T_U$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_tu.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]
        
#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[9][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Temperature Anomaly "+r'$\delta T_U$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_tu.png",dpi=300)   

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,sol.y[9][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Temperature Anomaly "+r'$\delta T_U$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_tu.png",dpi=300)

<IPython.core.display.Javascript object>

#### 2.2 $\delta T_I$

In [187]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        ax.plot(sol.t+1750,sol.y[10],c=targetColors[i],zorder = Nexp-i)

    plt.title('Temperature Anomaly '+ r'$\delta T_I$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$\delta T_I$ [°C]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_ti.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]
        
#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[10][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Temperature Anomaly "+r'$\delta T_I$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$\delta T_I$ [°C]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_ti.png",dpi=300)   

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_A_VIC[i]
        
        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[10][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Temperature Anomaly "+r'$\delta T_I$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$\delta T_I$ [°C]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_ti.png",dpi=300)

<IPython.core.display.Javascript object>

#### 2.3 $\delta T_D$

In [188]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        ax.plot(sol.t+1750,sol.y[11],c=targetColors[i],zorder = Nexp-i)

    plt.title('Temperature Anomaly '+ r'$\delta T_D$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$\delta T_D$ [°C]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_td.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]
        
#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[11][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Temperature Anomaly "+r'$\delta T_D$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$\delta T_D$ [°C]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_td.png",dpi=300)   

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_A_VIC[i]
        
        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[11][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Temperature Anomaly "+r'$\delta T_D$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$\delta T_D$ [°C]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_td.png",dpi=300)

<IPython.core.display.Javascript object>

#### 2.4 $M_A, \delta T_U, \delta T_I, \delta T_D$

In [189]:
fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_A_VIC[i]
    M_A_ppm = gtc_to_ppm(sol.y[0])
    ax1.plot(sol.t+1750,M_A_ppm,c=targetColors[i],ls='-')
    ax2.plot(sol.t+1750,sol.y[9],c=targetColors[i],ls='-')
    ax3.plot(sol.t+1750,sol.y[10],c=targetColors[i],ls='-')
    ax4.plot(sol.t+1750,sol.y[11],c=targetColors[i],ls='-')

plt.suptitle('Temperature Anomaly and '+r'$CO_2$'+" Concentration",fontsize=22)
ax1.set_ylabel(r'$M_A$ [ppm]',fontsize=14)
ax2.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)
ax3.set_ylabel(r'$\delta T_I$ [°C]',fontsize=14)
ax4.set_ylabel(r'$\delta T_D$ [°C]',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)
ax4.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_4.png", dpi=300)

<IPython.core.display.Javascript object>

### 3. Sea Level Rise 

#### 3.1 $S_{tot}$

In [190]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        ax.plot(sol.t+1750,SLR(sol),c=targetColors[i],zorder = Nexp-i)

    plt.title('Sea Level Rise ',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'SLR [m]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_slr.png",dpi=300)

# elif sim_100000==True:
    # gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
    # fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

    # time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    # for i in range(0,Nexp) :
        
    #     sol = solutions_CONF_A_VIC[i]
        
    #     for axes in [ax1,ax2,ax3] : 
            
    #             axes.plot(sol.t[time]+1750,SLR(sol)[time],c=targetColors[i],ls='-')
            
                        
    # ax1.set_xlim((1750,2300))
    # ax2.set_xlim((2300,10000))
    # ax3.set_xlim((10000,100000))
    # ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    # #for axes in [ax1,ax2,ax3,ax4] : 
    # #    axes.tick_params(labelbottom=False)
    # for axes in [ax2,ax3] : 
    #     axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
    #     axes.tick_params(labelleft=False)

    # plt.suptitle("Sea Level Rise",fontsize=20)
    # ax1.set_xticks([1750,1950,2100])
    # ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    # ax1.yaxis.set_tick_params(labelsize=13)
    # ax2.set_xticks([2300,5000,7500,10000])
    # ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    # ax3.set_xticks([25000,50000,75000,100000])
    # ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

    # ax1.set_ylabel(r'SLR [m]',fontsize=14)
    # ax2.set_xlabel(' Years AD',fontsize=14)

    # #fig.subplots_adjust(wspace=0.05) 
    # #fig.subplots_adjust(hspace=0.05) 

    # plt.tight_layout()
    # fig.subplots_adjust(wspace=0.05) 

    # fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_slr.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_A_VIC[i]
        
        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,SLR(sol)[time],c=targetColors[i],ls='-')
            
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Sea Level Rise",fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'SLR [m]',fontsize=14)
    ax2.set_xlabel('                            Years AD',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_slr.png",dpi=300)

<IPython.core.display.Javascript object>

#### 3.2 $\delta T_U, S_{tot}, S_{th}, S_{gl}, S_{AIS}, S_{GIS}$

In [191]:
fig,[[ax1,ax2],[ax3,ax4],[ax5,ax6]] = plt.subplots(3,2,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_A_VIC[i]

    ax1.plot(sol.t+1750,sol.y[9],c=targetColors[i],ls='-')
    ax2.plot(sol.t+1750,SLR(sol),c=targetColors[i],ls='-')
    ax3.plot(sol.t+1750,SLR_th(sol),c=targetColors[i],ls='-')
    ax4.plot(sol.t+1750,SLR_gl(sol),c=targetColors[i],ls='-')
    ax5.plot(sol.t+1750,SLR_AIS(sol),c=targetColors[i],ls='-')
    ax6.plot(sol.t+1750,SLR_GIS(sol),c=targetColors[i],ls='-')

plt.suptitle('Composition of Sea Level Rise',fontsize=22)
ax1.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)
ax2.set_ylabel(r'$S_{tot}$ [m]',fontsize=14)
ax3.set_ylabel(r'$S_{th}$ [m]',fontsize=14)
ax4.set_ylabel(r'$S_{gl}$ [m]',fontsize=14)
ax5.set_ylabel(r'$S_{AIS}$ [m]',fontsize=14)
ax6.set_ylabel(r'$S_{GIS}$ [m]',fontsize=14)

ax5.set_xlabel('Years AD',fontsize=14)
ax6.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_slr_comp.png", dpi=300)

<IPython.core.display.Javascript object>

### 4. $\delta SLR / \delta T_U$

In [192]:
fig,ax = plt.subplots(1,1,figsize=(8,8))

if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True or sim_100000==True:
    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        ax.plot(SLR(sol),sol.y[9],c=targetColors[i],zorder = Nexp-i)
    
elif sim_500000==True:
    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        ax.plot(SLR(sol)[time],sol.y[9][time],c=targetColors[i],zorder = Nexp-i)

plt.title(r"$(\delta SLR, \delta T_U)$",fontsize=26)
ax.tick_params(axis='both',labelsize=18)
ax.set_xlabel(r'$\delta$ SLR [m]',fontsize=18)
ax.set_ylabel(r'$\delta T_U$ [°C]',fontsize=18)

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_delta_slr_delta_t.png",dpi=300)

<IPython.core.display.Javascript object>

### 5. Tipping Elements States 

#### 5.1 $GIS,AMOC,WAIS,EASB,EAIS,AWSI,AMAZ,PERM$

In [193]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:

    fig,[[ax0,ax1],[ax2,ax3],[ax4,ax5],[ax6,ax7]] = plt.subplots(4,2,figsize=(12,6),sharex= True)

    for i in range(0,Nexp):

        sol = solutions_CONF_A_VIC[i]
        ax0.plot(sol.t+1750, sol.y[15],c=targetColors[i])
        ax1.plot(sol.t+1750, sol.y[16],c=targetColors[i])
        ax2.plot(sol.t+1750, sol.y[17],c=targetColors[i])
        ax3.plot(sol.t+1750, sol.y[18],c=targetColors[i])
        ax4.plot(sol.t+1750, sol.y[19],c=targetColors[i])
        ax5.plot(sol.t+1750, sol.y[20],c=targetColors[i])
        ax6.plot(sol.t+1750, sol.y[21],c=targetColors[i])
        ax7.plot(sol.t+1750, sol.y[22],c=targetColors[i])

    axes = [ax0,ax1,ax2,ax3,ax4,ax5,ax6,ax7]    

    for i in range(0,8):
        axes[i].set_title(labels[i])
        #axes[i].set_ylim([0,1])

    ax4.set_ylabel('                States of Tipping Elements [Adim]',fontsize=14)
    ax6.set_xlabel('Years AD',fontsize=14)
    ax7.set_xlabel('Years AD',fontsize=14)
        
    #ax0.set_xlim((10000,100000))
    plt.suptitle("States of Tipping Elements",fontsize=20)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_te_8states.png",dpi=400)
# elif sim_500000==True:
#     fig,[[ax0,ax1],[ax2,ax3],[ax4,ax5],[ax6,ax7]] = plt.subplots(4,2,figsize=(12,6),sharex= True)
      # time = ? # Definition of time to plot until 100 000 with a logarithmic scale
#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]
            
#         ax0.semilogx(sol.t[time]+1750, sol.y[15][time],c=targetColors[i])
#         ax1.semilogx(sol.t[time]+1750, sol.y[16][time],c=targetColors[i])
#         ax2.semilogx(sol.t[time]+1750, sol.y[17][time],c=targetColors[i])
#         ax3.semilogx(sol.t[time]+1750, sol.y[18][time],c=targetColors[i])
#         ax4.semilogx(sol.t[time]+1750, sol.y[19][time],c=targetColors[i])
#         ax5.semilogx(sol.t[time]+1750, sol.y[20][time],c=targetColors[i])
#         ax6.semilogx(sol.t[time]+1750, sol.y[21][time],c=targetColors[i])
#         ax7.semilogx(sol.t[time]+1750, sol.y[22][time],c=targetColors[i])

#     axes = [ax0,ax1,ax2,ax3,ax4,ax5,ax6,ax7]    

#     for i in range(0,8):
#         axes[i].set_title(labels[i])

#     ax4.set_ylabel('                States of Tipping Elements [Adim]',fontsize=14)
#     ax6.set_xlabel('Years AD',fontsize=14)
#     ax7.set_xlabel('Years AD',fontsize=14)
        
#     #ax0.set_xlim((10000,100000))
#     plt.suptitle("States of Tipping Elements",fontsize=20)
#     plt.tight_layout()

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_te_8states.png",dpi=400)

elif sim_500000==True:
    fig,[[ax0,ax1],[ax2,ax3],[ax4,ax5],[ax6,ax7]] = plt.subplots(4,2,figsize=(12,6),sharex= True)

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_A_VIC[i]
            
        ax0.semilogx(sol.t[time]+1750, sol.y[15][time],c=targetColors[i])
        ax1.semilogx(sol.t[time]+1750, sol.y[16][time],c=targetColors[i])
        ax2.semilogx(sol.t[time]+1750, sol.y[17][time],c=targetColors[i])
        ax3.semilogx(sol.t[time]+1750, sol.y[18][time],c=targetColors[i])
        ax4.semilogx(sol.t[time]+1750, sol.y[19][time],c=targetColors[i])
        ax5.semilogx(sol.t[time]+1750, sol.y[20][time],c=targetColors[i])
        ax6.semilogx(sol.t[time]+1750, sol.y[21][time],c=targetColors[i])
        ax7.semilogx(sol.t[time]+1750, sol.y[22][time],c=targetColors[i])

    axes = [ax0,ax1,ax2,ax3,ax4,ax5,ax6,ax7]    

    for i in range(0,8):
        axes[i].set_title(labels[i])

    ax4.set_ylabel('                States of Tipping Elements [Adim]',fontsize=14)
    ax6.set_xlabel('Years AD',fontsize=14)
    ax7.set_xlabel('Years AD',fontsize=14)
        
    #ax0.set_xlim((10000,100000))
    plt.suptitle("States of Tipping Elements",fontsize=20)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_te_8states.png",dpi=400)

<IPython.core.display.Javascript object>

#### 5.2 $AMOC,GIS$

In [194]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:

    fig,[ax0,ax1] = plt.subplots(2,1,figsize=(12,6),sharex= True)

    for i in range(0,Nexp):

        sol = solutions_CONF_A_VIC[i]
        ax0.plot(sol.t+1750, sol.y[16],c=targetColors[i])
        ax1.plot(sol.t+1750, sol.y[15],c=targetColors[i])

    ax0.set_title(labels[1])
    ax1.set_title(labels[0])
    ax0.tick_params(axis='both',labelsize=12)
    ax1.tick_params(axis='x',labelsize=15)
    ax1.set_ylabel('                                        States of Tipping Elements [Adim]',fontsize=14)
    ax1.set_xlabel('Years AD',fontsize=14)
        
    #ax0.set_xlim((10000,100000))
    plt.suptitle("States of AMOC and GIS",fontsize=20)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_te_2states.png",dpi=400)

# elif sim_100000==True:
#     fig,[ax0,ax1] = plt.subplots(2,1,figsize=(12,6),sharex= True)

#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]
            
#         ax0.semilogx(sol.t[time]+1750, sol.y[16][time],c=targetColors[i])
#         ax1.semilogx(sol.t[time]+1750, sol.y[15][time],c=targetColors[i])

#     ax0.set_title(labels[1])
#     ax1.set_title(labels[0])
#     ax0.tick_params(axis='both',labelsize=12)
#     ax1.tick_params(axis='x',labelsize=15)
#     ax1.set_ylabel('                                        States of Tipping Elements [Adim]',fontsize=14)
#     ax1.set_xlabel('Years AD',fontsize=14)
        
#     #ax0.set_xlim((10000,100000))
#     plt.suptitle("States of AMOC and GIS",fontsize=20)
#     plt.tight_layout()

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_te_2states.png",dpi=400)

elif sim_500000==True:
    fig,[ax0,ax1] = plt.subplots(2,1,figsize=(12,6),sharex= True)

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_A_VIC[i]
            
        ax0.semilogx(sol.t[time]+1750, sol.y[16][time],c=targetColors[i])
        ax1.semilogx(sol.t[time]+1750, sol.y[15][time],c=targetColors[i])

    ax0.set_title(labels[1])
    ax1.set_title(labels[0])
    ax0.tick_params(axis='both',labelsize=12)
    ax1.tick_params(axis='x',labelsize=15)
    ax1.set_ylabel('                                        States of Tipping Elements [Adim]',fontsize=14)
    ax1.set_xlabel('Years AD',fontsize=14)
        
    #ax0.set_xlim((10000,100000))
    plt.suptitle("States of AMOC and GIS",fontsize=20)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_te_2states.png",dpi=400)

<IPython.core.display.Javascript object>

#### 5.3 $AMOC, \delta T_U, F_{GIS}, F_{O}$

In [195]:
fig,[[ax0,ax1],[ax2,ax3]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

for i in range(0,Nexp):

    sol = solutions_CONF_A_VIC[i]
    sol_fgis = solutions_CONF_A_VIC_f[i][0]
    sol_fo = solutions_CONF_A_VIC_f[i][3]
    time = np.linspace(t0,tend,len(sol_fgis))
    ax0.plot(sol.t+1750, sol.y[16],c=targetColors[i])
    ax1.plot(sol.t+1750, sol.y[9],c=targetColors[i])
    ax2.plot(time+1750, sol_fgis,c=targetColors[i])
    ax3.plot(time+1750, sol_fo,c=targetColors[i])

ax0.set_ylabel(r'$\Psi \; [Adim]$',fontsize=14)
ax1.set_ylabel(r'$\delta T_U \;[°C]$ ',fontsize=14)
ax2.set_ylabel(r'$F_{GIS} \;[Sv]$ ',fontsize=14)
ax3.set_ylabel(r'$F_{O} \;[Sv]$ ',fontsize=14)

ax2.set_xlabel('Years AD',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)

# ax0.tick_params(axis='both',labelsize=12)
# ax1.tick_params(axis='x',labelsize=15)


    
#ax0.set_xlim((10000,100000))
plt.suptitle("States of AMOC and Forcing Variables",fontsize=20)
plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_te_amoc.png",dpi=400)

<IPython.core.display.Javascript object>

#### 5.4 $GIS, AMOC, \delta T_U$

In [196]:
fig,[ax0,ax1,ax2] = plt.subplots(3,1,figsize=(7,6),sharex= True)

for i in range(0,Nexp):

    sol = solutions_CONF_A_VIC[i]
    ax0.plot(sol.t+1750, sol.y[15],c=targetColors[i])
    ax1.plot(sol.t+1750, sol.y[16],c=targetColors[i])
    ax2.plot(sol.t+1750, sol.y[9],c=targetColors[i])

ax0.set_ylabel(r'$V \; [Adim]$',fontsize=14)
ax1.set_ylabel(r'$\Psi \;[Adim]$ ',fontsize=14)
ax2.set_ylabel(r'$\delta T_U \;[°C]$ ',fontsize=14)

ax2.set_xlabel('Years AD',fontsize=14)

# ax0.tick_params(axis='both',labelsize=12)
# ax1.tick_params(axis='x',labelsize=15)


    
#ax0.set_xlim((10000,100000))
plt.suptitle("States of GIS and Forcing Variables",fontsize=20)
plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_te_gis.png",dpi=400)

<IPython.core.display.Javascript object>

### 6. Freshwater Forcings 

#### 6.1 $F_{GIS}$

In [197]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        sol_fgis = solutions_CONF_A_VIC_f[i][0]
        time = np.linspace(t0,tend,len(sol_fgis))
        ax.plot(time+1750,sol_fgis,c=targetColors[i],zorder = Nexp-i)

    plt.title('Freshwater Forcing '+ r'$F_{GIS}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$F_{GIS}$ [Sv]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fgis.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]

#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[23][time],c=targetColors[i],ls='-')
            
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)
        
#         plt.suptitle("Freshwater Forcing "+r'$F_{GIS}}$',fontsize=20)
#         ax1.set_xticks([1750,1950,2100])
#         ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#         ax1.yaxis.set_tick_params(labelsize=13)
#         ax2.set_xticks([2300,5000,7500,10000])
#         ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#         ax3.set_xticks([25000,50000,75000,100000])
#         ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
#         ax4.set_xticks([250000,500000])
#         ax4.set_xticklabels(['250k','500k'],fontsize=12)

#         ax1.set_ylabel(r'$F_{GIS}$ [Sv]',fontsize=14)

#     ax2.set_xlabel(' Years AD',fontsize=14)
#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fgis.png")

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time_log = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


    for i in range(0,Nexp) :
        
        sol = solutions_CONF_A_VIC[i]
        sol_fgis = solutions_CONF_A_VIC_fgis[i][0]
        time = np.linspace(t0,tend,len(sol_fgis))

        for axes in [ax1,ax2,ax3,ax4] : 
                ax.plot(time[time_log]+1750,sol_fgis[time_log],c=targetColors[i],zorder = Nexp-i)    
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))



    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)
        
        plt.suptitle("Freshwater Forcing "+r'$F_{GIS}}$',fontsize=20)
        ax1.set_xticks([1750,1950,2100])
        ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
        ax1.yaxis.set_tick_params(labelsize=13)
        ax2.set_xticks([2300,5000,7500,10000])
        ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
        ax3.set_xticks([25000,50000,75000,100000])
        ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
        ax4.set_xticks([250000,500000])
        ax4.set_xticklabels(['250k','500k'],fontsize=12)

        ax1.set_ylabel(r'$F_{GIS}$ [Sv]',fontsize=14)

    ax2.set_xlabel('                            Years AD',fontsize=14)
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fgis.png",dpi=300)

<IPython.core.display.Javascript object>

#### 6.2 $F_{W}$

In [198]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        sol_fw = solutions_CONF_A_VIC_f[i][1]
        time = np.linspace(t0,tend,len(sol_fw))
        ax.plot(time+1750,sol_fw,c=targetColors[i],zorder = Nexp-i)

    plt.title('Freshwater Forcing '+ r'$F_{W}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$F_{W}$ [Sv]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fw.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]

#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[24][time],c=targetColors[i],ls='-')
            
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)
        
#         plt.suptitle("Freshwater Forcing "+r'$F_{W}}$',fontsize=20)
#         ax1.set_xticks([1750,1950,2100])
#         ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#         ax1.yaxis.set_tick_params(labelsize=13)
#         ax2.set_xticks([2300,5000,7500,10000])
#         ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#         ax3.set_xticks([25000,50000,75000,100000])
#         ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
#         ax4.set_xticks([250000,500000])
#         ax4.set_xticklabels(['250k','500k'],fontsize=12)

#         ax1.set_ylabel(r'$F_{W}$ [Adim]',fontsize=14)

#     ax2.set_xlabel(' Years AD',fontsize=14)
#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fw.png")

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


    for i in range(0,Nexp) :
        
        sol = solutions_CONF_A_VIC[i]

        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[24][time],c=targetColors[i],ls='-')
            
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))

    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)
        
        plt.suptitle("Freshwater Forcing "+r'$F_{W}}$',fontsize=20)
        ax1.set_xticks([1750,1950,2100])
        ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
        ax1.yaxis.set_tick_params(labelsize=13)
        ax2.set_xticks([2300,5000,7500,10000])
        ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
        ax3.set_xticks([25000,50000,75000,100000])
        ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
        ax4.set_xticks([250000,500000])
        ax4.set_xticklabels(['250k','500k'],fontsize=12)

        ax1.set_ylabel(r'$F_{W}$ [Adim]',fontsize=14)

    ax2.set_xlabel('                            Years AD',fontsize=14)
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fw.png",dpi=300)

<IPython.core.display.Javascript object>

#### 6.3 $F_{E-P}$

In [199]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        sol_fep = solutions_CONF_A_VIC_f[i][2]
        time = np.linspace(t0,tend,len(sol_fep))
        ax.plot(time+1750,sol_fep,c=targetColors[i],zorder = Nexp-i)

    plt.title('Freshwater Forcing '+ r'$F_{E-P}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$F_{E-P}$ [Sv]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fep.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]

#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[25][time],c=targetColors[i],ls='-')
            
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)
        
#         plt.suptitle("Freshwater Forcing "+r'$F_{E-P}}$',fontsize=20)
#         ax1.set_xticks([1750,1950,2100])
#         ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#         ax1.yaxis.set_tick_params(labelsize=13)
#         ax2.set_xticks([2300,5000,7500,10000])
#         ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#         ax3.set_xticks([25000,50000,75000,100000])
#         ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
#         ax4.set_xticks([250000,500000])
#         ax4.set_xticklabels(['250k','500k'],fontsize=12)

#         ax1.set_ylabel(r'$F_{E-P}$ [Adim]',fontsize=14)

#     ax2.set_xlabel(' Years AD',fontsize=14)
#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fep.png")

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


    for i in range(0,Nexp) :
        
        sol = solutions_CONF_A_VIC[i]

        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[25][time],c=targetColors[i],ls='-')
            
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))

    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)
        
        plt.suptitle("Freshwater Forcing "+r'$F_{E-P}}$',fontsize=20)
        ax1.set_xticks([1750,1950,2100])
        ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
        ax1.yaxis.set_tick_params(labelsize=13)
        ax2.set_xticks([2300,5000,7500,10000])
        ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
        ax3.set_xticks([25000,50000,75000,100000])
        ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
        ax4.set_xticks([250000,500000])
        ax4.set_xticklabels(['250k','500k'],fontsize=12)

        ax1.set_ylabel(r'$F_{E-P}$ [Adim]',fontsize=14)

    ax2.set_xlabel('                            Years AD',fontsize=14)
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fep.png",dpi=300)

<IPython.core.display.Javascript object>

#### 6.4 $F_O$

In [200]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        sol_fo = solutions_CONF_A_VIC_f[i][3]
        time = np.linspace(t0,tend,len(sol_fo))
        ax.plot(time+1750,sol_fo,c=targetColors[i],zorder = Nexp-i)

    plt.title('Freshwater Forcing '+ r'$F_{O}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$F_{O}$ [Sv]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fo.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]

#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[26][time],c=targetColors[i],ls='-')
            
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)
        
#         plt.suptitle("Freshwater Forcing "+r'$F_{O}}$',fontsize=20)
#         ax1.set_xticks([1750,1950,2100])
#         ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#         ax1.yaxis.set_tick_params(labelsize=13)
#         ax2.set_xticks([2300,5000,7500,10000])
#         ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#         ax3.set_xticks([25000,50000,75000,100000])
#         ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
#         ax4.set_xticks([250000,500000])
#         ax4.set_xticklabels(['250k','500k'],fontsize=12)

#         ax1.set_ylabel(r'$F_{O}$ [Adim]',fontsize=14)

#     ax2.set_xlabel(' Years AD',fontsize=14)
#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fo.png")

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


    for i in range(0,Nexp) :
        
        sol = solutions_CONF_A_VIC[i]

        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[26][time],c=targetColors[i],ls='-')
            
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))

    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)
        
        plt.suptitle("Freshwater Forcing "+r'$F_{O}}$',fontsize=20)
        ax1.set_xticks([1750,1950,2100])
        ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
        ax1.yaxis.set_tick_params(labelsize=13)
        ax2.set_xticks([2300,5000,7500,10000])
        ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
        ax3.set_xticks([25000,50000,75000,100000])
        ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
        ax4.set_xticks([250000,500000])
        ax4.set_xticklabels(['250k','500k'],fontsize=12)

        ax1.set_ylabel(r'$F_{O}$ [Adim]',fontsize=14)

    ax2.set_xlabel('                            Years AD',fontsize=14)
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_fo.png",dpi=300)

<IPython.core.display.Javascript object>

#### 6.5 $F_{GIS}, F_O, F_W, F_{E-P}$

In [201]:
fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_A_VIC[i]
    sol_fgis = solutions_CONF_A_VIC_f[i][0]
    sol_fo = solutions_CONF_A_VIC_f[i][3]
    sol_fw = solutions_CONF_A_VIC_f[i][1]
    sol_fep = solutions_CONF_A_VIC_f[i][2]
    time = np.linspace(t0,tend,len(sol_fgis))

    ax1.plot(time+1750,sol_fgis,c=targetColors[i],ls='-')
    ax2.plot(time+1750,sol_fo,c=targetColors[i],ls='-')
    ax3.plot(time+1750,sol_fw,c=targetColors[i],ls='-')
    ax4.plot(time+1750,sol_fep,c=targetColors[i],ls='-')

plt.suptitle('Freshwater Flux Forcings',fontsize=22)
ax1.set_ylabel(r'$F_{GIS}$ [Sv]',fontsize=14)
ax2.set_ylabel(r'$F_O$ [Sv]',fontsize=14)
ax3.set_ylabel(r'$F_W$ [Sv]',fontsize=14)
ax4.set_ylabel(r'$F_{E-P}$ [Sv]',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)
ax4.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_f_4.png", dpi=300)

<IPython.core.display.Javascript object>

### 7. Carbon Cycle  

#### 7.1 $M_A,M_U,M_I,M_D$

In [202]:
fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

for i in range (0, Nexp):
    sol = solutions_CONF_A_VIC[i]

    ax1.plot(sol.t+1750,sol.y[0],c=targetColors[i],ls='-')
    ax2.plot(sol.t+1750,sol.y[2],c=targetColors[i],ls='-')
    ax3.plot(sol.t+1750,sol.y[3],c=targetColors[i],ls='-')
    ax4.plot(sol.t+1750,sol.y[4],c=targetColors[i],ls='-')

plt.suptitle('Carbon Reservoirs '+r'$M_A,M_U,M_I,M_D$',fontsize=22)
ax1.set_ylabel(r'$M_A$ [PgC]',fontsize=14)
ax2.set_ylabel(r'$M_U$ [PgC]',fontsize=14)
ax3.set_ylabel(r'$M_I$ [PgC]',fontsize=14)
ax4.set_ylabel(r'$M_D$ [PgC]',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)
ax4.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_4.png", dpi=300)

<IPython.core.display.Javascript object>

#### 7.2 $M_{CH_4}$

In [203]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))
    conv_ppm=True

    for i in range(0,Nexp):
        sol = solutions_CONF_A_VIC[i]
        if conv_ppm == True :
            M_CH4_ppm = gtc_to_ppm(sol.y[12])
            ax.plot(sol.t+1750,M_CH4_ppm,c=targetColors[i],zorder = Nexp-i)
        else:
            ax.plot(sol.t+1750,sol.y[12],c=targetColors[i],zorder = Nexp-i)

    plt.title('Carbon Reservoir '+ r'$M_{CH_4}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    if conv_ppm==True:
        ax.set_ylabel(r'$M_{CH_4}$ [ppm]',fontsize=14)
    else:
        ax.set_ylabel(r'$M_{CH_4}$ [PgC]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    if conv_ppm == True :
        fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_mch4_ppm.png",dpi=300)
    else: 
        fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_mch4.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_A_VIC[i]
#         M_CH4_ppm = gtc_to_ppm(sol.y[12])
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,M_CH4_ppm,c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_{CH_4}$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_{CH_4}$ [PgC',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_mch4_ppm.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        M_CH4_ppm = gtc_to_ppm(sol.y[12])
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,M_CH4_ppm,c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_{CH_4}$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_{CH_4}$ [PgC]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_mch4_ppm.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.3 $M_L$

In [204]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        ax.plot(sol.t+1750,sol.y[1],c=targetColors[i],zorder = Nexp-i)

    plt.title('Carbon Reservoir '+ r'$M_{L}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$M_{L}$ [PgC]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_l.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_A_VIC[i]
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,sol.y[1][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_{L}$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_{L}$ [PgC]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_temp_ml.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,sol.y[1][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_{L}$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_{L}$ [PgC]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_l.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.4 $M_A$

In [205]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        M_A_ppm = gtc_to_ppm(sol.y[0])
        ax.plot(sol.t+1750,M_A_ppm,c=targetColors[i],zorder = Nexp-i)

    plt.title('Atmospheric '+r'$CO_2$'+' Concentration',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$M_A$ [ppm]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_a_ppm.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_A_VIC[i]
#         M_A_ppm = gtc_to_ppm(sol.y[0])
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,M_A_ppm,c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_A$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_A$ [ppm]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_a_ppm.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        M_A_ppm = gtc_to_ppm(sol.y[0])
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,M_A_ppm,c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_A$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_A$ [PgC]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_a_ppm.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.5 $M_U$

In [206]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        ax.plot(sol.t+1750,sol.y[2],c=targetColors[i],zorder = Nexp-i)

    plt.title('Carbon Reservoir '+ r'$M_U$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$M_U$ [PgC]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_u.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_A_VIC[i]
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,sol.y[2][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_U$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_U$ [PgC]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_u.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,sol.y[2][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_U$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_U$ [PgC]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_u.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.6 $M_D$

In [207]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        ax.plot(sol.t+1750,sol.y[4],c=targetColors[i],zorder = Nexp-i)

    plt.title('Carbon Reservoir '+ r'$M_D$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$M_D$ [PgC]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_d.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_A_VIC[i]
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,sol.y[4][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_D$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_D$ [PgC]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_d.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_A_VIC[i]
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,sol.y[4][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_D$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_D$ [PgC]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_d.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.7 $k_{UI}, k_{IU}, k_{ID}, k_{DI}$

In [208]:
fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol_carb = solutions_CONF_A_VIC_carb_impact_param[i]
    k_ui = sol_carb[1]
    k_iu = sol_carb[2]
    k_id = sol_carb[3]
    k_di = sol_carb[4]
    sol = solutions_CONF_A_VIC[i]
    time = np.linspace(t0,tend,len(k_ui))

    ax1.plot(time+1750,k_ui,c=targetColors[i],ls='-')
    ax2.plot(time+1750,k_iu,c=targetColors[i],ls='-')
    ax3.plot(time+1750,k_id,c=targetColors[i],ls='-')
    ax4.plot(time+1750,k_di,c=targetColors[i],ls='-')

plt.suptitle('Carbon Flux Coefficients '+r'$k_{UI}, k_{IU}, k_{ID}, k_{DI}$',fontsize=22)
ax1.set_ylabel(r'$k_{UI} [s^{-1}]$',fontsize=14)
ax2.set_ylabel(r'$k_{IU} [s^{-1}]$',fontsize=14)
ax3.set_ylabel(r'$k_{ID} [s^{-1}]$',fontsize=14)
ax4.set_ylabel(r'$k_{DI} [s^{-1}]$',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)
ax4.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_k_4.png", dpi=300)

<IPython.core.display.Javascript object>

#### 7.8 $k_{AU}$

In [209]:
fig,ax1 = plt.subplots(1,1,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol_carb = solutions_CONF_A_VIC_carb_impact_param[i]
    k_au = sol_carb[0]
    sol = solutions_CONF_A_VIC[i]
    time = np.linspace(t0,tend,len(k_au))

    ax1.plot(time+1750,k_au,c=targetColors[i],ls='-')


plt.suptitle('Carbon Flux Coefficient '+r'$k_{AU}$',fontsize=22)
ax1.set_ylabel(r'$k_{AU} [s^{-1}]$',fontsize=14)
ax.tick_params(axis='both',labelsize=13)
ax1.set_xlabel('Years AD',fontsize=14)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_carb_k_au.png", dpi=300)

<IPython.core.display.Javascript object>

### 8. $pH$

#### 8.1 $pH_U$

In [210]:
fig,ax1 = plt.subplots(1,1,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_A_VIC[i]
    ph = -np.log10(species(sol,T_U,S_U,ktemp=True,level='U')[0])
    time = np.linspace(t0,tend,len(ph))

    ax1.plot(time+1750,ph,c=targetColors[i],ls='-')


plt.suptitle('Oceanic Acidity of the Upper layer '+r'$pH_U$',fontsize=22)
ax1.set_ylabel(r'$pH$',fontsize=14)
ax.tick_params(axis='both',labelsize=13)
ax1.set_xlabel('Years AD',fontsize=14)


plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_pH.png", dpi=300)

<IPython.core.display.Javascript object>

#### 8.2 $pH_U, pH_I, pH,D$

In [211]:
fig,[ax1,ax2,ax3] = plt.subplots(3,1,figsize=(12,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_A_VIC[i]
    ph_u = -np.log10(species(sol,T_U,S_U,ktemp=True,level='U')[0])
    ph_i = -np.log10(species(sol,T_I,S_I,ktemp=True,level='I')[0])
    ph_d = -np.log10(species(sol,T_D,S_D,ktemp=True,level='D')[0])
    time = np.linspace(t0,tend,len(ph))

    ax1.plot(time+1750,ph_u,c=targetColors[i],ls='-')
    ax2.plot(time+1750,ph_i,c=targetColors[i],ls='-')
    ax3.plot(time+1750,ph_d,c=targetColors[i],ls='-')


plt.suptitle('Oceanic Acidity '+r'$pH_U, pH_I, pH_D$',fontsize=22)
ax1.set_ylabel(r'$pH_U$',fontsize=14)
ax2.set_ylabel(r'$pH_I$',fontsize=14)
ax3.set_ylabel(r'$pH_D$',fontsize=14)
ax.tick_params(axis='both',labelsize=13)
ax3.set_xlabel('Years AD',fontsize=14)


plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_pH_3.png", dpi=300)

<IPython.core.display.Javascript object>

---
# Simulations - CONF_B
---
CONF_B_VIC  = SSPs scenarios on SURFER v4.0 with configuration with : 
- Worst case coupling in the Tipping Point Model 
- Activated interactions between the tipping elements 
- Activated AMOC-GIS Tipping Cascade Calibration Module 
- Activated AMOC Collapse Impacts on carbon cycle 

## Setting Up

###  AGTCCM  

In [212]:
AGTCCM = True

if AGTCCM == False:
    adv_param_amoc_vic = False

###  cGenie Calibration for AMOC

In [213]:
if AGTCCM == True:
    cGenie_calib = True 
if AGTCCM == False:
    cGenie_calib = False

###  Methane cycle

In [214]:
meth_te_emi = True 
meth_fossil_emi = False 

### Tipping Element Coupling and Feedback

Same as CONFA

### Worst Case Coupling of Tipping Element States

In [215]:
##### Critical Values for Tipping Elements and coefficient computation #####

### 0 - GIS ###

## Retrieval of the calibration coefficients based on the AGTCCM or not ##
if AGTCCM==True:
    Tp_GIS = 0.8
    Tm_GIS = 0.3
    Vp_GIS = 0.77
    taup_GIS = 1000
    taum_GIS = 1000
    # Definition of the other critical parameters # 
    Vm_GIS = xm_today(Tp_GIS, Tm_GIS, Vp_GIS) # Lower critical value of the GIS volume [Adim]
    Psip_GIS = 0.7 # Higher critical AMOC's intensity value for the GIS (Non defined in Couplet (2023))
    Psim_GIS = 0.1 # Lower critical AMOC's intensity value for the GIS (Non defined in Couplet (2023))
    # Definition of the sensibility experiment parameters #
    Psi_C = 0.9 # Constant value of the AMOC's intensity for the sensibility experiment C [Adim]
    TGIS_D = 1.5 # Constant value of the temperature for the sensibility experiment D [°C]
    # Computation of the calibration coefficients based on AGTCCM # 
    GIS_Module_calib_coeff = calib_module_GIS(Vp_GIS,Vm_GIS,Tp_GIS,Tm_GIS,Psip_GIS,Psim_GIS,Psi_C,TGIS_D)
    a_GIS = GIS_Module_calib_coeff[0]
    b_GIS = GIS_Module_calib_coeff[1]
    c_GIS = GIS_Module_calib_coeff[4]
    d_GIS = GIS_Module_calib_coeff[2]
    e_GIS = GIS_Module_calib_coeff[3]
if AGTCCM==False:
    Tp_GIS = 0.8
    Tm_GIS = 0.3
    Vp_GIS = 0.77
    taup_GIS = 1000
    taum_GIS = 1000
    a2_GIS,a1_GIS,c1_GIS,c0_GIS = params(Tp_GIS,Tm_GIS,Vp_GIS)

### 1 - WAIS ###

Tp_WAIS = 1
Tm_WAIS = 0.3
Vp_WAIS = 0.75
taup_WAIS = 500
taum_WAIS = 500
a2_WAIS,a1_WAIS,c1_WAIS,c0_WAIS = params(Tp_WAIS,Tm_WAIS,Vp_WAIS)

### 2 - EASB ###

Tp_EASB = 2
Tm_EASB = 0.3
Vp_EASB = 0.75
taup_EASB = 500
taum_EASB = 500
a2_EASB,a1_EASB,c1_EASB,c0_EASB = params(Tp_EASB,Tm_EASB,Vp_EASB)

### 3 - EAIS ###

Tp_EAIS = 5
Tm_EAIS = 2.0
Vp_EAIS = 0.44
taup_EAIS = 10000
taum_EAIS = 10000
a2_EAIS,a1_EAIS,c1_EAIS,c0_EAIS = params(Tp_EAIS,Tm_EAIS,Vp_EAIS)

### 4 - AWSI ###

Tp_AWSI = 4.5
Tm_AWSI = 2.5
Vp_AWSI = 0.7
taup_AWSI = 10
taum_AWSI = 10
a2_AWSI,a1_AWSI,c1_AWSI,c0_AWSI = params(Tp_AWSI,Tm_AWSI,Vp_AWSI)

### 5 - AMOC ###

## Retrieval of the calibration coefficients based on the AGTCCM or not ##
if AGTCCM==True:

    if cGenie_calib == False : 
        Tp_AMOC = 1.4
        Tm_AMOC = 0
        Psip_AMOC = 0.77
        taup_AMOC = 15
        taum_AMOC = 15
        # Definition of the other critical parameters # 
        Psim_AMOC = xm_today(Tp_AMOC, Tm_AMOC, Psip_AMOC) # Lower critical value of the AMOC Intensity [Adim]
        FGISp_AMOC = 0.5 # Higher critical fresh water forcing value from the GIS for the AMOC (Non defined in Couplet (2023))
        FGISm_AMOC = 0.05 # Lower critical fresh water forcing value from the GIS for the AMOC (Non defined in Couplet (2023))
        FOp_AMOC = 0.8 # Higher critical fresh water forcing value from the other components for the AMOC (Non defined in Couplet (2023))
        FOm_AMOC = 0.08 # Lower critical fresh water forcing value from the other components for the AMOC (Non defined in Couplet (2023))
        # Definition of the sensibility experiment parameters #
        FGIS_A = 0.8 # Constant value of the fresh water flux from the GIS for the sensibility experiment A [Sv] 
        FO_A = 0.3 # Constant value of the fresh water flux from the Other components for the sensibility experiment A [Sv]
        TAMOC_B = 2 # Constant value of the temperature for the sensibility experiment B [°C]
        FO_B = 0.8 # Constant value of the fresh water flux from the Other components for the sensibility experiment B [Sv]
        TAMOC_C = 2 # Constant value of the temperature for the sensibility experiment C [°C]
        FGIS_C = 0.5 # Constant value of the fresh water flux from the GIS for the sensibility experiment C [Sv] 
        # Computation of the calibration coefficients based on AGTCCM # 
        AMOC_Module_calib_coeff= calib_module_AMOC_ParamB(Psip_AMOC,Psim_AMOC,Tp_AMOC,Tm_AMOC,FGISp_AMOC,FGISm_AMOC,FOp_AMOC,FOm_AMOC,FGIS_A,FO_A,TAMOC_B,FO_B,TAMOC_C,FGIS_C)
        a_AMOC = AMOC_Module_calib_coeff[0]
        b_AMOC = AMOC_Module_calib_coeff[1]
        c_AMOC = AMOC_Module_calib_coeff[5]
        d_AMOC = AMOC_Module_calib_coeff[2]
        e_AMOC = AMOC_Module_calib_coeff[3]
        f_AMOC = AMOC_Module_calib_coeff[4]

    if cGenie_calib == True:
        # Definition of the other critical parameters # 
        Psip_AMOC = 0.6
        Psim_AMOC = 0.022
        Tp_AMOC = 5.5
        Tm_AMOC = 1.27
        FGISp_AMOC = 0.03
        FGISm_AMOC = -0.1
        FOp_AMOC = 0.065
        FOm_AMOC = -0.02
        taup_AMOC = 50
        taum_AMOC = 50
        # Definition of the sensibility experiment parameters #
        F_GIS_cst_A = 0
        F_O_cst_A = 0
        T_forcing_cst_B = 0
        F_O_cst_B = 0
        T_forcing_cst_C = 0
        F_GIS_cst_C = 0
        # Computation of the calibration coefficients based on AGTCCM # 
        AMOC_Module_calib_coeff = calib_module_AMOC_ParamB(Psip_AMOC,Psim_AMOC,Tp_AMOC,Tm_AMOC,FGISp_AMOC,FGISm_AMOC,FOp_AMOC,FOm_AMOC,F_GIS_cst_A, F_O_cst_A, T_forcing_cst_B, F_O_cst_B, T_forcing_cst_C, F_GIS_cst_C)
        a_AMOC = AMOC_Module_calib_coeff[0]
        b_AMOC = AMOC_Module_calib_coeff[1]
        c_AMOC = AMOC_Module_calib_coeff[5]
        c_bis1 = AMOC_Module_calib_coeff[6]
        c_bis2 = AMOC_Module_calib_coeff[7]
        d_AMOC = AMOC_Module_calib_coeff[2]
        e_AMOC = AMOC_Module_calib_coeff[3]
        f_AMOC = AMOC_Module_calib_coeff[4]

if AGTCCM==False:
    Tp_AMOC = 1.4
    Tm_AMOC = 0
    Psip_AMOC = 0.77
    taup_AMOC = 15
    taum_AMOC = 15
    a2_AMOC,a1_AMOC,c1_AMOC,c0_AMOC = params(Tp_AMOC,Tm_AMOC,Psip_AMOC)


### 6 - AMAZ ###

Tp_AMAZ = 2
Tm_AMAZ = 0
Vp_AMAZ = 0.77
taup_AMAZ = 50
taum_AMAZ = 50
a2_AMAZ,a1_AMAZ,c1_AMAZ,c0_AMAZ = params(Tp_AMAZ,Tm_AMAZ,Vp_AMAZ)

### 7 - PERM ###

Tp_PERM = 3
Tm_PERM = 0
Vp_PERM = 0.75
taup_PERM = 10
taum_PERM = 10
a2_PERM,a1_PERM,c1_PERM,c0_PERM = params(Tp_PERM,Tm_PERM,Vp_PERM)

### SLR ###

Spot_GIS = 7
Spot_WAIS = 5
Spot_EASB = 9
Spot_EAIS = 45

##### Vectors of the coefficients and parameters #####

if AGTCCM==False:
    A2 = np.array([a2_GIS,a2_AMOC,a2_WAIS,a2_EASB,a2_EAIS,a2_AWSI,a2_AMAZ,a2_PERM])
    A1 = np.array([a1_GIS,a1_AMOC,a1_WAIS,a1_EASB,a1_EAIS,a1_AWSI,a1_AMAZ,a1_PERM])
    C1 = np.array([c1_GIS,c1_AMOC,c1_WAIS,c1_EASB,c1_EAIS,c1_AWSI,c1_AMAZ,c1_PERM])
    C0 = np.array([c0_GIS,c0_AMOC,c0_WAIS,c0_EASB,c0_EAIS,c0_AWSI,c0_AMAZ,c0_PERM])
if AGTCCM==True:
    A2_AGTCCM = np.array([a2_WAIS,a2_EASB,a2_EAIS,a2_AWSI,a2_AMAZ,a2_PERM])
    A1_AGTCCM = np.array([a1_WAIS,a1_EASB,a1_EAIS,a1_AWSI,a1_AMAZ,a1_PERM])
    C1_AGTCCM = np.array([c1_WAIS,c1_EASB,c1_EAIS,c1_AWSI,c1_AMAZ,c1_PERM])
    C0_AGTCCM = np.array([c0_WAIS,c0_EASB,c0_EAIS,c0_AWSI,c0_AMAZ,c0_PERM])

TAUP = np.array([taup_GIS,taup_AMOC,taup_WAIS,taup_EASB,taup_EAIS,taup_AWSI,taup_AMAZ,taup_PERM])
TAUM = np.array([taum_GIS,taum_AMOC,taum_WAIS,taum_EASB,taum_EAIS,taum_AWSI,taum_AMAZ,taum_PERM])

FMOD = np.array([fmod_GIS,fmod_AMOC,fmod_WAIS,fmod_EASB,fmod_EAIS,fmod_AWSI,fmod_AMAZ,fmod_PERM])
CPOT = np.array([cpot_GIS,cpot_AMOC,cpot_WAIS,cpot_EASB,cpot_EAIS,cpot_AWSI,cpot_AMAZ,cpot_PERM])
CH4POT = np.array([ch4pot_GIS,ch4pot_AMOC,ch4pot_WAIS,ch4pot_EASB,ch4pot_EAIS,ch4pot_AWSI,ch4pot_AMAZ,ch4pot_PERM])

### Interactions between Tipping Elements

In [216]:
cfeedback = 1
d= 0.5
epsi = 1

### Freshwater Forcing 

In [217]:
if AGTCCM == True:
    f_w_forc = True
    f_ep_forc = False
if AGTCCM == False:
    f_w_forc = False
    f_ep_forc = False

### AMOC Collapse Impacts on Carbon Cycle

In [218]:
### Activation of the AMOC impacts on carbon cycle model ### 
if AGTCCM==True:
    amoc_impact_carb = True

    amoc_impact_carb_au = False # Was used for test but has to remain on False due to physical reasons

    impact_k_ui = True
    impact_k_iu = True

    amoc_impact_carb_id = True
    impact_k_id = True
    impact_k_di = True

    ## Calibration coefficients for the impacts on carbon cycle model ##
    c_Sto = -0.3
    c_mean_o = 0.01
    c_mean_a = 0.3

    c_AU = c_Sto*c_mean_a

    c_UI = c_Sto*c_mean_o
    c_IU = c_Sto*c_mean_o

    c_ID = c_Sto*c_mean_o
    c_DI = c_Sto*c_mean_o

if AGTCCM ==False:
    amoc_impact_carb = False

    amoc_impact_carb_au = False # Was used for test but has to remain on False due to physical reasons

    impact_k_ui = False
    impact_k_iu = False

    amoc_impact_carb_id = False
    impact_k_id = False
    impact_k_di = False

## Simulations

In [219]:
                                        ##### Simulations Parameters #####
### Choosing Number of Experiences and Scenarios ###
Nexp = 15

if sim_2300==True:
    tend = 550
    save_fig_path2='/SIM 2300/'
elif sim_2500==True: 
    tend = 750
    save_fig_path2='/SIM 2500/'
elif sim_5000==True:
    tend = 3250
    save_fig_path2='/SIM 5000/'
elif sim_10000==True:
    tend = 8250
    save_fig_path2='/SIM 10 000/'
elif sim_100000==True:
    tend = 98250
    save_fig_path2='/SIM 100 000/'
elif sim_500000==True:
    tend=500000
    save_fig_path2='/SIM 500 000/'

t0=0 # Initial time 
dt=1 # Time step 
n = int((tend-t0)/dt)+1 # Number of time steps 
teval=np.linspace(t0,tend,n) # List of discrete time at which solutions are stored

                                            ##### Initial State ######
if cGenie_calib==True:
    psi_0 = 0.9284
else:
    psi_0 = 1
y0 = [MA_pi,ML_pi,MU_pi,MI_pi,MD_pi,QU_pi,QI_pi,QD_pi,MS_pi,0,0,0,Mch4_pi,ML_pi,0,1,psi_0,1,1,1,1,1,1,0,0,0,0] # Initial Vector State

co2_landuse = zero_forcing(teval) # Land use CO2 parameterisation 
ch4_landuse = zero_forcing(teval) # Land use CH2 parameterisation 
if meth_fossil_emi==False:
    ch4_fossil = zero_forcing(teval) # Fossil CH4 parameterisation

                                        ##### Solutions Computations #####

tot_emit_scen = np.linspace(1000,5000,Nexp) # List with the different total CO2 emissions in PgC that will be used to create the emission scenarios
solutions_CONF_B_VIC = [] # Array to store the solutions 
solutions_CONF_B_VIC_carb_impact_param = [] # Array to store the solutions of the coefficient of carbon flux
solutions_CONF_B_VIC_f = [] # Array to store the solutions related to the freshwater fluxes

sourceColors = ["#0077BB", "#CC3311",] # Color interpolator 
targetColors = colInterpolatOr(sourceColors, colorCode = "HEX", nTarget = Nexp)

for i in range(0,Nexp) : 
    
    co2_fossil = forcingCO2(2010,tot_emit_scen[i],mode='total',landuse= True, origin=1750) 
    sol, sol_carb, sol_f = SURFER_T(co2_fossil,co2_landuse,ch4_fossil,ch4_landuse,y0,t0,tend,dt,ktemp = True,sediments = True ,weathering = True,atmtoland_flux='old', int_method = 'LSODA')
    solutions_CONF_B_VIC.append(sol)
    solutions_CONF_B_VIC_carb_impact_param.append(sol_carb)
    solutions_CONF_B_VIC_f.append(sol_f)

    print("Scenario {} with : {:.2f} PgC of total emission done".format(i+1,tot_emit_scen[i])) 

Scenario 1 with : 1000.00 PgC of total emission done


Scenario 2 with : 1285.71 PgC of total emission done
Scenario 3 with : 1571.43 PgC of total emission done
Scenario 4 with : 1857.14 PgC of total emission done
Scenario 5 with : 2142.86 PgC of total emission done
Scenario 6 with : 2428.57 PgC of total emission done
Scenario 7 with : 2714.29 PgC of total emission done
Scenario 8 with : 3000.00 PgC of total emission done
Scenario 9 with : 3285.71 PgC of total emission done
Scenario 10 with : 3571.43 PgC of total emission done
Scenario 11 with : 3857.14 PgC of total emission done
Scenario 12 with : 4142.86 PgC of total emission done
Scenario 13 with : 4428.57 PgC of total emission done
Scenario 14 with : 4714.29 PgC of total emission done
Scenario 15 with : 5000.00 PgC of total emission done


## Results

### 1. Bifurcation Diagram

In [220]:
colors = ['C0','C1','C2','C3','C4','C5','C6','C7']

fig, ax = plt.subplots(1,1,figsize=(8,5))

if AGTCCM == False:

    labels = ['GIS','AMOC','WAIS','EASB','EAIS','AWSI','AMAZ','PERM']

    for i in range(0,8):
        
        x = np.linspace(-0.8,1.2,1000)
        y = -1*(-x**3+A2[i]*x**2+A1[i]*x+C0[i])/C1[i]

        ax.plot(y[x>0],x[x>0],c=colors[i],label = labels[i])
        ax.plot([y[x>0][0],17],[0,0],c=colors[i])
        
    ax.set_xlim((-0.5,10))
    ax.legend(fontsize=12)

    ax.set_xlabel('T (°C)')
    ax.set_ylabel('Tipping element state')

if AGTCCM == True:

    labels = ['GIS','AMOC','WAIS','EASB','EAIS','AWSI','AMAZ','PERM']

    x = np.linspace(-0.8,1.2,1000)

    ## Plot of the bifurcation diagram for GIS and AMOC ## 

    yGIS = (x**3-a_GIS*x**2-b_GIS*x-c_GIS-e_GIS*(1-Psi_C))/d_GIS
    yAMOC = (x**3-a_AMOC*x**2-b_AMOC*x-c_AMOC-e_AMOC*F_GIS_cst_A-f_AMOC*F_O_cst_A)/d_AMOC

    ax.plot(yGIS[x>0],x[x>0],c=colors[0],label = labels[0])
    ax.plot(yAMOC[x>0],x[x>0],c=colors[1],label = labels[1])

    ax.plot([yGIS[x>0][0],17],[0,0],c=colors[0])
    ax.plot([yAMOC[x>0][0],17],[0,0],c=colors[1])

    ## Plot of the bifurcation diagram for all the others Tipping Elements ##  
    for i in range(0,6):
        x = np.linspace(-0.8,1.2,1000)
        y = -1*(-x**3+A2_AGTCCM[i]*x**2+A1_AGTCCM[i]*x+C0_AGTCCM[i])/C1_AGTCCM[i]

        ax.plot(y[x>0],x[x>0],c=colors[i+2],label = labels[i+2])
        ax.plot([y[x>0][0],17],[0,0],c=colors[i+2])
        
    ax.set_xlim((-0.5,10))
    ax.legend(fontsize=12)

    ax.set_xlabel('T [°C]',fontsize=14)
    ax.set_ylabel('State of the Tipping Elements [Adim]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.title("Bifurcation Diagrams",fontsize=20)

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_bifdiag.png",dpi=300)

<IPython.core.display.Javascript object>

### 2. Temperature

#### 2.1 $\delta T_U$

In [221]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        ax.plot(sol.t+1750,sol.y[9],c=targetColors[i],zorder = Nexp-i)

    plt.title('Temperature Anomaly '+ r'$\delta T_U$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_tu.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_B_VIC[i]
        
#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[9][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Temperature Anomaly "+r'$\delta T_U$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_tu.png",dpi=300)   

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]
        
        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[9][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Temperature Anomaly "+r'$\delta T_U$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_tu.png",dpi=300)

<IPython.core.display.Javascript object>

#### 2.2 $\delta T_I$

In [222]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        ax.plot(sol.t+1750,sol.y[10],c=targetColors[i],zorder = Nexp-i)

    plt.title('Temperature Anomaly '+ r'$\delta T_I$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$\delta T_I$ [°C]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_ti.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_B_VIC[i]
        
#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[10][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Temperature Anomaly "+r'$\delta T_I$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$\delta T_I$ [°C]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_ti.png",dpi=300)   

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]
        
        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[10][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Temperature Anomaly "+r'$\delta T_I$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$\delta T_I$ [°C]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_ti.png",dpi=300)

<IPython.core.display.Javascript object>

#### 2.3 $\delta T_D$

In [223]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        ax.plot(sol.t+1750,sol.y[11],c=targetColors[i],zorder = Nexp-i)

    plt.title('Temperature Anomaly '+ r'$\delta T_D$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$\delta T_D$ [°C]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_td.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_B_VIC[i]
        
#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[11][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Temperature Anomaly "+r'$\delta T_D$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$\delta T_D$ [°C]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_td.png",dpi=300)   

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]
        
        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[11][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Temperature Anomaly "+r'$\delta T_D$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$\delta T_D$ [°C]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_td.png",dpi=300)

<IPython.core.display.Javascript object>

#### 2.4 $M_A, \delta T_U, \delta T_I, \delta T_D$

In [224]:
fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_B_VIC[i]
    M_A_ppm = gtc_to_ppm(sol.y[0])
    ax1.plot(sol.t+1750,M_A_ppm,c=targetColors[i],ls='-')
    ax2.plot(sol.t+1750,sol.y[9],c=targetColors[i],ls='-')
    ax3.plot(sol.t+1750,sol.y[10],c=targetColors[i],ls='-')
    ax4.plot(sol.t+1750,sol.y[11],c=targetColors[i],ls='-')

plt.suptitle('Temperature Anomaly and '+r'$CO_2$'+" Concentration",fontsize=22)
ax1.set_ylabel(r'$M_A$ [ppm]',fontsize=14)
ax2.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)
ax3.set_ylabel(r'$\delta T_I$ [°C]',fontsize=14)
ax4.set_ylabel(r'$\delta T_D$ [°C]',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)
ax4.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_4.png", dpi=300)

<IPython.core.display.Javascript object>

### 3. Sea Level Rise 

#### 3.1 $S_{tot}$

In [225]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        ax.plot(sol.t+1750,SLR(sol),c=targetColors[i],zorder = Nexp-i)

    plt.title('Sea Level Rise ',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'SLR (m)',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_slr.png",dpi=300)

# elif sim_100000==True:
    # gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
    # fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

    # time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    # for i in range(0,Nexp) :
        
    #     sol = solutions_CONF_B_VIC[i]
        
    #     for axes in [ax1,ax2,ax3] : 
            
    #             axes.plot(sol.t[time]+1750,SLR(sol)[time],c=targetColors[i],ls='-')
            
                        
    # ax1.set_xlim((1750,2300))
    # ax2.set_xlim((2300,10000))
    # ax3.set_xlim((10000,100000))
    # ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    # #for axes in [ax1,ax2,ax3,ax4] : 
    # #    axes.tick_params(labelbottom=False)
    # for axes in [ax2,ax3] : 
    #     axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
    #     axes.tick_params(labelleft=False)

    # plt.suptitle("Sea Level Rise",fontsize=20)
    # ax1.set_xticks([1750,1950,2100])
    # ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    # ax1.yaxis.set_tick_params(labelsize=13)
    # ax2.set_xticks([2300,5000,7500,10000])
    # ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    # ax3.set_xticks([25000,50000,75000,100000])
    # ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

    # ax1.set_ylabel(r'SLR (m)',fontsize=14)
    # ax2.set_xlabel(' Years AD',fontsize=14)

    # #fig.subplots_adjust(wspace=0.05) 
    # #fig.subplots_adjust(hspace=0.05) 

    # plt.tight_layout()
    # fig.subplots_adjust(wspace=0.05) 

    # fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_slr.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]
        
        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,SLR(sol)[time],c=targetColors[i],ls='-')
            
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Sea Level Rise",fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'SLR (m)',fontsize=14)
    ax2.set_xlabel('                            Years AD',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_slr.png",dpi=300)

<IPython.core.display.Javascript object>

#### 3.2 $\delta T_U, S_{tot}, S_{th}, S_{gl}, S_{AIS}, S_{GIS}$

In [226]:
fig,[[ax1,ax2],[ax3,ax4],[ax5,ax6]] = plt.subplots(3,2,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_B_VIC[i]

    ax1.plot(sol.t+1750,sol.y[9],c=targetColors[i],ls='-')
    ax2.plot(sol.t+1750,SLR(sol),c=targetColors[i],ls='-')
    ax3.plot(sol.t+1750,SLR_th(sol),c=targetColors[i],ls='-')
    ax4.plot(sol.t+1750,SLR_gl(sol),c=targetColors[i],ls='-')
    ax5.plot(sol.t+1750,SLR_AIS(sol),c=targetColors[i],ls='-')
    ax6.plot(sol.t+1750,SLR_GIS(sol),c=targetColors[i],ls='-')

plt.suptitle('Composition of Sea Level Rise',fontsize=22)
ax1.set_ylabel(r'$\delta T_U$ [°C]',fontsize=14)
ax2.set_ylabel(r'$S_{tot}$ [m]',fontsize=14)
ax3.set_ylabel(r'$S_{th}$ [m]',fontsize=14)
ax4.set_ylabel(r'$S_{gl}$ [m]',fontsize=14)
ax5.set_ylabel(r'$S_{AIS}$ [m]',fontsize=14)
ax6.set_ylabel(r'$S_{GIS}$ [m]',fontsize=14)

ax5.set_xlabel('Years AD',fontsize=14)
ax6.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_slr_comp.png", dpi=300)

<IPython.core.display.Javascript object>

### 4. $\delta SLR / \delta T_U$

In [227]:
fig,ax = plt.subplots(1,1,figsize=(8,8))

if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True or sim_100000==True:
    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        ax.plot(SLR(sol),sol.y[9],c=targetColors[i],zorder = Nexp-i)
    
elif sim_500000==True:
    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        ax.plot(SLR(sol)[time],sol.y[9][time],c=targetColors[i],zorder = Nexp-i)

plt.title(r"$(\delta SLR, \delta T_U)$",fontsize=26)
ax.tick_params(axis='both',labelsize=18)
ax.set_xlabel(r'$\delta$ SLR [m]',fontsize=18)
ax.set_ylabel(r'$\delta T_U$ [°C]',fontsize=18)

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_delta_slr_delta_t.png",dpi=300)

<IPython.core.display.Javascript object>

### 5. Tipping Elements States

#### 5.1 $GIS,AMOC,WAIS,EASB,EAIS,AWSI,AMAZ,PERM$

In [228]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:

    fig,[[ax0,ax1],[ax2,ax3],[ax4,ax5],[ax6,ax7]] = plt.subplots(4,2,figsize=(12,6),sharex= True)

    for i in range(0,Nexp):

        sol = solutions_CONF_B_VIC[i]
        ax0.plot(sol.t+1750, sol.y[15],c=targetColors[i])
        ax1.plot(sol.t+1750, sol.y[16],c=targetColors[i])
        ax2.plot(sol.t+1750, sol.y[17],c=targetColors[i])
        ax3.plot(sol.t+1750, sol.y[18],c=targetColors[i])
        ax4.plot(sol.t+1750, sol.y[19],c=targetColors[i])
        ax5.plot(sol.t+1750, sol.y[20],c=targetColors[i])
        ax6.plot(sol.t+1750, sol.y[21],c=targetColors[i])
        ax7.plot(sol.t+1750, sol.y[22],c=targetColors[i])

    axes = [ax0,ax1,ax2,ax3,ax4,ax5,ax6,ax7]    

    for i in range(0,8):
        axes[i].set_title(labels[i])
        #axes[i].set_ylim([0,1])

    ax4.set_ylabel('                States of Tipping Elements [Adim]',fontsize=14)
    ax6.set_xlabel('Years AD',fontsize=14)
    ax7.set_xlabel('Years AD',fontsize=14)
        
    #ax0.set_xlim((10000,100000))
    plt.suptitle("States of Tipping Elements",fontsize=20)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_te_8states.png",dpi=400)
# elif sim_500000==True:
#     fig,[[ax0,ax1],[ax2,ax3],[ax4,ax5],[ax6,ax7]] = plt.subplots(4,2,figsize=(12,6),sharex= True)
      # time = ? # Definition of time to plot until 100 000 with a logarithmic scale
#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_A_VIC[i]
            
#         ax0.semilogx(sol.t[time]+1750, sol.y[15][time],c=targetColors[i])
#         ax1.semilogx(sol.t[time]+1750, sol.y[16][time],c=targetColors[i])
#         ax2.semilogx(sol.t[time]+1750, sol.y[17][time],c=targetColors[i])
#         ax3.semilogx(sol.t[time]+1750, sol.y[18][time],c=targetColors[i])
#         ax4.semilogx(sol.t[time]+1750, sol.y[19][time],c=targetColors[i])
#         ax5.semilogx(sol.t[time]+1750, sol.y[20][time],c=targetColors[i])
#         ax6.semilogx(sol.t[time]+1750, sol.y[21][time],c=targetColors[i])
#         ax7.semilogx(sol.t[time]+1750, sol.y[22][time],c=targetColors[i])

#     axes = [ax0,ax1,ax2,ax3,ax4,ax5,ax6,ax7]    

#     for i in range(0,8):
#         axes[i].set_title(labels[i])

#     ax4.set_ylabel('                States of Tipping Elements [Adim]',fontsize=14)
#     ax6.set_xlabel('Years AD',fontsize=14)
#     ax7.set_xlabel('Years AD',fontsize=14)
        
#     #ax0.set_xlim((10000,100000))
#     plt.suptitle("States of Tipping Elements",fontsize=20)
#     plt.tight_layout()

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_A_VIC_te_8states.png",dpi=400)

elif sim_500000==True:
    fig,[[ax0,ax1],[ax2,ax3],[ax4,ax5],[ax6,ax7]] = plt.subplots(4,2,figsize=(12,6),sharex= True)

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]
            
        ax0.semilogx(sol.t[time]+1750, sol.y[15][time],c=targetColors[i])
        ax1.semilogx(sol.t[time]+1750, sol.y[16][time],c=targetColors[i])
        ax2.semilogx(sol.t[time]+1750, sol.y[17][time],c=targetColors[i])
        ax3.semilogx(sol.t[time]+1750, sol.y[18][time],c=targetColors[i])
        ax4.semilogx(sol.t[time]+1750, sol.y[19][time],c=targetColors[i])
        ax5.semilogx(sol.t[time]+1750, sol.y[20][time],c=targetColors[i])
        ax6.semilogx(sol.t[time]+1750, sol.y[21][time],c=targetColors[i])
        ax7.semilogx(sol.t[time]+1750, sol.y[22][time],c=targetColors[i])

    axes = [ax0,ax1,ax2,ax3,ax4,ax5,ax6,ax7]    

    for i in range(0,8):
        axes[i].set_title(labels[i])

    ax4.set_ylabel('                States of Tipping Elements [Adim]',fontsize=14)
    ax6.set_xlabel('Years AD',fontsize=14)
    ax7.set_xlabel('Years AD',fontsize=14)
        
    #ax0.set_xlim((10000,100000))
    plt.suptitle("States of Tipping Elements",fontsize=20)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_te_8states.png",dpi=400)

<IPython.core.display.Javascript object>

#### 5.2 $AMOC, GIS$

In [229]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:

    fig,[ax0,ax1] = plt.subplots(2,1,figsize=(12,6),sharex= True)

    for i in range(0,Nexp):

        sol = solutions_CONF_B_VIC[i]
        ax0.plot(sol.t+1750, sol.y[16],c=targetColors[i])
        ax1.plot(sol.t+1750, sol.y[15],c=targetColors[i])

    ax0.set_title(labels[1])
    ax1.set_title(labels[0])
    ax0.tick_params(axis='both',labelsize=12)
    ax1.tick_params(axis='x',labelsize=15)
    ax1.set_ylabel('                                        States of Tipping Elements [Adim]',fontsize=14)
    ax1.set_xlabel('Years AD',fontsize=14)
        
    #ax0.set_xlim((10000,100000))
    plt.suptitle("States of AMOC and GIS",fontsize=20)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_te_2states.png",dpi=400)

# elif sim_100000==True:
#     fig,[ax0,ax1] = plt.subplots(2,1,figsize=(12,6),sharex= True)

#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_B_VIC[i]
            
#         ax0.semilogx(sol.t[time]+1750, sol.y[16][time],c=targetColors[i])
#         ax1.semilogx(sol.t[time]+1750, sol.y[15][time],c=targetColors[i])

#     ax0.set_title(labels[1])
#     ax1.set_title(labels[0])
#     ax0.tick_params(axis='both',labelsize=12)
#     ax1.tick_params(axis='x',labelsize=15)
#     ax1.set_ylabel('                                        States of Tipping Elements [Adim]',fontsize=14)
#     ax1.set_xlabel('Years AD',fontsize=14)
        
#     #ax0.set_xlim((10000,100000))
#     plt.suptitle("States of AMOC and GIS",fontsize=20)
#     plt.tight_layout()

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_te_2states.png",dpi=400)

elif sim_500000==True:
    fig,[ax0,ax1] = plt.subplots(2,1,figsize=(12,6),sharex= True)

    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]
            
        ax0.semilogx(sol.t[time]+1750, sol.y[16][time],c=targetColors[i])
        ax1.semilogx(sol.t[time]+1750, sol.y[15][time],c=targetColors[i])

    ax0.set_title(labels[1])
    ax1.set_title(labels[0])
    ax0.tick_params(axis='both',labelsize=12)
    ax1.tick_params(axis='x',labelsize=15)
    ax1.set_ylabel('                                        States of Tipping Elements [Adim]',fontsize=14)
    ax1.set_xlabel('Years AD',fontsize=14)
        
    #ax0.set_xlim((10000,100000))
    plt.suptitle("States of AMOC and GIS",fontsize=20)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_te_2states.png",dpi=400)

<IPython.core.display.Javascript object>

#### 5.3 $AMOC, \delta T_U, F_{GIS}, F_{O}$

In [230]:
fig,[[ax0,ax1],[ax2,ax3]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

for i in range(0,Nexp):

    sol = solutions_CONF_B_VIC[i]
    sol_fgis = solutions_CONF_B_VIC_f[i][0]
    sol_fo = solutions_CONF_B_VIC_f[i][3]
    time = np.linspace(t0,tend,len(sol_fgis))
    ax0.plot(sol.t+1750, sol.y[16],c=targetColors[i])
    ax1.plot(sol.t+1750, sol.y[9],c=targetColors[i])
    ax2.plot(time+1750, sol_fgis,c=targetColors[i])
    ax3.plot(time+1750, sol_fo,c=targetColors[i])

ax0.set_ylabel(r'$\Psi \; [Adim]$',fontsize=14)
ax1.set_ylabel(r'$\delta T_U \;[°C]$ ',fontsize=14)
ax2.set_ylabel(r'$F_{GIS} \;[Sv]$ ',fontsize=14)
ax3.set_ylabel(r'$F_{O} \;[Sv]$ ',fontsize=14)

ax2.set_xlabel('Years AD',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)

# ax0.tick_params(axis='both',labelsize=12)
# ax1.tick_params(axis='x',labelsize=15)


    
#ax0.set_xlim((10000,100000))
plt.suptitle("States of AMOC and Forcing Variables",fontsize=20)
plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_te_amoc.png",dpi=400)

<IPython.core.display.Javascript object>

#### 5.4 $GIS, AMOC, \delta T_U$

In [231]:
fig,[ax0,ax1,ax2] = plt.subplots(3,1,figsize=(7,6),sharex= True)

for i in range(0,Nexp):

    sol = solutions_CONF_B_VIC[i]
    ax0.plot(sol.t+1750, sol.y[15],c=targetColors[i])
    ax1.plot(sol.t+1750, sol.y[16],c=targetColors[i])
    ax2.plot(sol.t+1750, sol.y[9],c=targetColors[i])

ax0.set_ylabel(r'$V \; [Adim]$',fontsize=14)
ax1.set_ylabel(r'$\Psi \;[Adim]$ ',fontsize=14)
ax2.set_ylabel(r'$\delta T_U \;[°C]$ ',fontsize=14)

ax2.set_xlabel('Years AD',fontsize=14)

# ax0.tick_params(axis='both',labelsize=12)
# ax1.tick_params(axis='x',labelsize=15)


    
#ax0.set_xlim((10000,100000))
plt.suptitle("States of GIS and Forcing Variables",fontsize=20)
plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_te_gis.png",dpi=400)

<IPython.core.display.Javascript object>

### 6. Freshwater Forcings

#### 6.1 $F_{GIS}$

In [232]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        sol_fgis = solutions_CONF_B_VIC_f[i][0]
        time = np.linspace(t0,tend,len(sol_fgis))
        ax.plot(time+1750,sol_fgis,c=targetColors[i],zorder = Nexp-i)

    plt.title('Freshwater Forcing '+ r'$F_{GIS}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$F_{GIS}$ [Sv]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fgis.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_B_VIC[i]

#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[23][time],c=targetColors[i],ls='-')
            
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)
        
#         plt.suptitle("Freshwater Forcing "+r'$F_{GIS}}$',fontsize=20)
#         ax1.set_xticks([1750,1950,2100])
#         ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#         ax1.yaxis.set_tick_params(labelsize=13)
#         ax2.set_xticks([2300,5000,7500,10000])
#         ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#         ax3.set_xticks([25000,50000,75000,100000])
#         ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
#         ax4.set_xticks([250000,500000])
#         ax4.set_xticklabels(['250k','500k'],fontsize=12)

#         ax1.set_ylabel(r'$F_{GIS}$ [Sv]',fontsize=14)

#     ax2.set_xlabel(' Years AD',fontsize=14)
#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fgis.png")

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time_log = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]
        sol_fgis = solutions_CONF_B_VIC_f[i][0]
        time = np.linspace(t0,tend,len(sol_fgis))

        for axes in [ax1,ax2,ax3,ax4] : 
                ax.plot(time[time_log]+1750,sol_fgis[time_log],c=targetColors[i],zorder = Nexp-i)    
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))



    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)
        
        plt.suptitle("Freshwater Forcing "+r'$F_{GIS}}$',fontsize=20)
        ax1.set_xticks([1750,1950,2100])
        ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
        ax1.yaxis.set_tick_params(labelsize=13)
        ax2.set_xticks([2300,5000,7500,10000])
        ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
        ax3.set_xticks([25000,50000,75000,100000])
        ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
        ax4.set_xticks([250000,500000])
        ax4.set_xticklabels(['250k','500k'],fontsize=12)

        ax1.set_ylabel(r'$F_{GIS}$ [Sv]',fontsize=14)

    ax2.set_xlabel('                            Years AD',fontsize=14)
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fgis.png",dpi=300)

<IPython.core.display.Javascript object>

#### 6.2 $F_{W}$

In [233]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        sol_fw = solutions_CONF_B_VIC_f[i][1]
        time = np.linspace(t0,tend,len(sol_fw))
        ax.plot(time+1750,sol_fw,c=targetColors[i],zorder = Nexp-i)

    plt.title('Freshwater Forcing '+ r'$F_{W}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$F_{W}$ [Sv]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fw.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_B_VIC[i]

#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[24][time],c=targetColors[i],ls='-')
            
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)
        
#         plt.suptitle("Freshwater Forcing "+r'$F_{W}}$',fontsize=20)
#         ax1.set_xticks([1750,1950,2100])
#         ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#         ax1.yaxis.set_tick_params(labelsize=13)
#         ax2.set_xticks([2300,5000,7500,10000])
#         ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#         ax3.set_xticks([25000,50000,75000,100000])
#         ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
#         ax4.set_xticks([250000,500000])
#         ax4.set_xticklabels(['250k','500k'],fontsize=12)

#         ax1.set_ylabel(r'$F_{W}$ [Adim]',fontsize=14)

#     ax2.set_xlabel(' Years AD',fontsize=14)
#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fw.png")

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]

        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[24][time],c=targetColors[i],ls='-')
            
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))

    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)
        
        plt.suptitle("Freshwater Forcing "+r'$F_{W}}$',fontsize=20)
        ax1.set_xticks([1750,1950,2100])
        ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
        ax1.yaxis.set_tick_params(labelsize=13)
        ax2.set_xticks([2300,5000,7500,10000])
        ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
        ax3.set_xticks([25000,50000,75000,100000])
        ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
        ax4.set_xticks([250000,500000])
        ax4.set_xticklabels(['250k','500k'],fontsize=12)

        ax1.set_ylabel(r'$F_{W}$ [Adim]',fontsize=14)

    ax2.set_xlabel('                            Years AD',fontsize=14)
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fw.png",dpi=300)

<IPython.core.display.Javascript object>

#### 6.3 $F_{E-P}$

In [234]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        sol_fep = solutions_CONF_B_VIC_f[i][2]
        time = np.linspace(t0,tend,len(sol_fep))
        ax.plot(time+1750,sol_fep,c=targetColors[i],zorder = Nexp-i)

    plt.title('Freshwater Forcing '+ r'$F_{E-P}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$F_{E-P}$ [Sv]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fep.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_B_VIC[i]

#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[25][time],c=targetColors[i],ls='-')
            
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)
        
#         plt.suptitle("Freshwater Forcing "+r'$F_{E-P}}$',fontsize=20)
#         ax1.set_xticks([1750,1950,2100])
#         ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#         ax1.yaxis.set_tick_params(labelsize=13)
#         ax2.set_xticks([2300,5000,7500,10000])
#         ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#         ax3.set_xticks([25000,50000,75000,100000])
#         ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
#         ax4.set_xticks([250000,500000])
#         ax4.set_xticklabels(['250k','500k'],fontsize=12)

#         ax1.set_ylabel(r'$F_{E-P}$ [Adim]',fontsize=14)

#     ax2.set_xlabel(' Years AD',fontsize=14)
#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fw.png")

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]

        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[25][time],c=targetColors[i],ls='-')
            
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))

    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)
        
        plt.suptitle("Freshwater Forcing "+r'$F_{E-P}}$',fontsize=20)
        ax1.set_xticks([1750,1950,2100])
        ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
        ax1.yaxis.set_tick_params(labelsize=13)
        ax2.set_xticks([2300,5000,7500,10000])
        ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
        ax3.set_xticks([25000,50000,75000,100000])
        ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
        ax4.set_xticks([250000,500000])
        ax4.set_xticklabels(['250k','500k'],fontsize=12)

        ax1.set_ylabel(r'$F_{E-P}$ [Adim]',fontsize=14)

    ax2.set_xlabel('                            Years AD',fontsize=14)
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fep.png",dpi=300)

<IPython.core.display.Javascript object>

#### 6.4 $F_O$

In [235]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        sol_fo = solutions_CONF_B_VIC_f[i][3]
        time = np.linspace(t0,tend,len(sol_fo))
        ax.plot(time+1750,sol_fo,c=targetColors[i],zorder = Nexp-i)

    plt.title('Freshwater Forcing '+ r'$F_{O}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$F_{O}$ [Sv]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fo.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


#     for i in range(0,Nexp) :
        
#         sol = solutions_CONF_B_VIC[i]

#         for axes in [ax1,ax2,ax3] : 
            
#                 axes.plot(sol.t[time]+1750,sol.y[26][time],c=targetColors[i],ls='-')
            
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))

#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top


#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)
        
#         plt.suptitle("Freshwater Forcing "+r'$F_{O}}$',fontsize=20)
#         ax1.set_xticks([1750,1950,2100])
#         ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#         ax1.yaxis.set_tick_params(labelsize=13)
#         ax2.set_xticks([2300,5000,7500,10000])
#         ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#         ax3.set_xticks([25000,50000,75000,100000])
#         ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
#         ax4.set_xticks([250000,500000])
#         ax4.set_xticklabels(['250k','500k'],fontsize=12)

#         ax1.set_ylabel(r'$F_{O}$ [Adim]',fontsize=14)

#     ax2.set_xlabel(' Years AD',fontsize=14)
#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fo.png")

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1


    for i in range(0,Nexp) :
        
        sol = solutions_CONF_B_VIC[i]

        for axes in [ax1,ax2,ax3,ax4] : 
            
                axes.plot(sol.t[time]+1750,sol.y[26][time],c=targetColors[i],ls='-')
            
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))

    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top

    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)
        
        plt.suptitle("Freshwater Forcing "+r'$F_{O}}$',fontsize=20)
        ax1.set_xticks([1750,1950,2100])
        ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
        ax1.yaxis.set_tick_params(labelsize=13)
        ax2.set_xticks([2300,5000,7500,10000])
        ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
        ax3.set_xticks([25000,50000,75000,100000])
        ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
        ax4.set_xticks([250000,500000])
        ax4.set_xticklabels(['250k','500k'],fontsize=12)

        ax1.set_ylabel(r'$F_{O}$ [Adim]',fontsize=14)

    ax2.set_xlabel('                            Years AD',fontsize=14)
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_fo.png",dpi=300)

<IPython.core.display.Javascript object>

#### 6.5 $F_{GIS}, F_O, F_W, F_{E-P}$

In [236]:
fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_B_VIC[i]
    sol_fgis = solutions_CONF_B_VIC_f[i][0]
    sol_fo = solutions_CONF_B_VIC_f[i][3]
    sol_fw = solutions_CONF_B_VIC_f[i][1]
    sol_fep = solutions_CONF_B_VIC_f[i][2]
    time = np.linspace(t0,tend,len(sol_fgis))

    ax1.plot(time+1750,sol_fgis,c=targetColors[i],ls='-')
    ax2.plot(time+1750,sol_fo,c=targetColors[i],ls='-')
    ax3.plot(time+1750,sol_fw,c=targetColors[i],ls='-')
    ax4.plot(time+1750,sol_fep,c=targetColors[i],ls='-')

plt.suptitle('Freshwater Flux Forcings',fontsize=22)
ax1.set_ylabel(r'$F_{GIS}$ [Sv]',fontsize=14)
ax2.set_ylabel(r'$F_O$ [Sv]',fontsize=14)
ax3.set_ylabel(r'$F_W$ [Sv]',fontsize=14)
ax4.set_ylabel(r'$F_{E-P}$ [Sv]',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)
ax4.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_f_4.png", dpi=300)

<IPython.core.display.Javascript object>

### 7. Carbon Cycle 

#### 7.1 $M_A,M_U,M_I,M_D$

In [237]:
fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

for i in range (0, Nexp):
    sol = solutions_CONF_B_VIC[i]

    ax1.plot(sol.t+1750,sol.y[0],c=targetColors[i],ls='-')
    ax2.plot(sol.t+1750,sol.y[2],c=targetColors[i],ls='-')
    ax3.plot(sol.t+1750,sol.y[3],c=targetColors[i],ls='-')
    ax4.plot(sol.t+1750,sol.y[4],c=targetColors[i],ls='-')

plt.suptitle('Carbon Reservoirs '+r'$M_A,M_U,M_I,M_D$',fontsize=22)
ax1.set_ylabel(r'$M_A$ [PgC]',fontsize=14)
ax2.set_ylabel(r'$M_U$ [PgC]',fontsize=14)
ax3.set_ylabel(r'$M_I$ [PgC]',fontsize=14)
ax4.set_ylabel(r'$M_D$ [PgC]',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)
ax4.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_4.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.2 $M_{CH_4}$

In [238]:
conv_ppm=False

if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        if conv_ppm == True :
            M_CH4_ppm = gtc_to_ppm(sol.y[12])
            ax.plot(sol.t+1750,M_CH4_ppm,c=targetColors[i],zorder = Nexp-i)
        else:
            ax.plot(sol.t+1750,sol.y[12],c=targetColors[i],zorder = Nexp-i)

    plt.title('Carbon Reservoir '+ r'$M_{CH_4}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    if conv_ppm==True:
        ax.set_ylabel(r'$M_{CH_4}$ [ppm]',fontsize=14)
    else:
        ax.set_ylabel(r'$M_{CH_4}$ [PgC]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    if conv_ppm == True :
        fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_mch4_ppm.png",dpi=300)
    else: 
        fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_mch4.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_B_VIC[i]
#         M_CH4_ppm = gtc_to_ppm(sol.y[12])
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,M_CH4_ppm,c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_{CH_4}$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_{CH_4}$ [PgC',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_temp_mch4_ppm.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        M_CH4_ppm = gtc_to_ppm(sol.y[12])
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,M_CH4_ppm,c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_{CH_4}$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_{CH_4}$ [PgC]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_mch4_ppm.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.3 $M_L$

In [239]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        ax.plot(sol.t+1750,sol.y[1],c=targetColors[i],zorder = Nexp-i)

    plt.title('Carbon Reservoir '+ r'$M_{L}$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$M_{L}$ [PgC]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_l.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_B_VIC[i]
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,sol.y[1][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_{L}$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_{L}$ [PgC]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_l.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,sol.y[1][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_{L}$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_{L}$ [PgC]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_l.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.4 $M_A$

In [240]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        M_A_ppm = gtc_to_ppm(sol.y[0])
        ax.plot(sol.t+1750,M_A_ppm,c=targetColors[i],zorder = Nexp-i)

    plt.title('Atmospheric '+r'$CO_2$'+' Concentration',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$M_A$ [ppm]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_a_ppm.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_B_VIC[i]
#         M_A_ppm = gtc_to_ppm(sol.y[0])
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,M_A_ppm,c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_A$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_A$ [ppm]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_a_ppm.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        M_A_ppm = gtc_to_ppm(sol.y[0])
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,M_A_ppm,c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_A$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_A$ [ppm]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_a_ppm.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.5 $M_U$

In [241]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        ax.plot(sol.t+1750,sol.y[2],c=targetColors[i],zorder = Nexp-i)

    plt.title('Carbon Reservoir '+ r'$M_U$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$M_U$ [PgC]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_u.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_B_VIC[i]
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,sol.y[2][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_U$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_U$ [PgC]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_u.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,sol.y[2][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_U$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_U$ [PgC]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_u.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.6 $M_D$

In [242]:
if sim_2300==True or sim_2500==True or sim_5000==True or sim_10000==True:
    fig,ax = plt.subplots(1,1,figsize=(8,6))

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        ax.plot(sol.t+1750,sol.y[4],c=targetColors[i],zorder = Nexp-i)

    plt.title('Carbon Reservoir '+ r'$M_D$',fontsize=22)
    ax.set_xlabel('Years AD',fontsize=14)
    ax.set_ylabel(r'$M_D$ [PgC]',fontsize=14)
    ax.tick_params(axis='both',labelsize=13)
    plt.tight_layout()

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_d.png",dpi=300)

# elif sim_100000==True:
#     gs_kw = dict(width_ratios=[1,1,1],height_ratios=[1])
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(10,4),gridspec_kw=gs_kw)

#     time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

#     for i in range(0,Nexp) :
#         sol = solutions_CONF_B_VIC[i]
#         for axes in [ax1,ax2,ax3] : 
#             axes.plot(sol.t[time]+1750,sol.y[4][time],c=targetColors[i],ls='-')
                        
#     ax1.set_xlim((1750,2300))
#     ax2.set_xlim((2300,10000))
#     ax3.set_xlim((10000,100000))
#     ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
#     #for axes in [ax1,ax2,ax3,ax4] : 
#     #    axes.tick_params(labelbottom=False)
#     for axes in [ax2,ax3] : 
#         axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
#         axes.tick_params(labelleft=False)

#     plt.suptitle("Carbon Reservoir "+r'$M_D$',fontsize=20)
#     ax1.set_xticks([1750,1950,2100])
#     ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
#     ax1.yaxis.set_tick_params(labelsize=13)
#     ax2.set_xticks([2300,5000,7500,10000])
#     ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
#     ax3.set_xticks([25000,50000,75000,100000])
#     ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)

#     ax1.set_ylabel(r'$M_D$ [PgC]',fontsize=14)

#     #fig.subplots_adjust(wspace=0.05) 
#     #fig.subplots_adjust(hspace=0.05) 

#     ax2.set_xlabel(' Years AD',fontsize=14)


#     plt.tight_layout()
#     fig.subplots_adjust(wspace=0.05) 

#     fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_d.png",dpi=300)

elif sim_500000==True:
    gs_kw = dict(width_ratios=[1,1,1,1],height_ratios=[1])
    fig, [ax1,ax2,ax3,ax4] = plt.subplots(1,4, figsize=(10,4),gridspec_kw=gs_kw)

    time = (np.rint(np.logspace(0,6-np.log10(2),1000))).astype(int)-1

    for i in range(0,Nexp) :
        sol = solutions_CONF_B_VIC[i]
        for axes in [ax1,ax2,ax3,ax4] : 
            axes.plot(sol.t[time]+1750,sol.y[4][time],c=targetColors[i],ls='-')
                        
    ax1.set_xlim((1750,2300))
    ax2.set_xlim((2300,10000))
    ax3.set_xlim((10000,100000))
    ax4.set_xlim((100000,500000))
    ax1.tick_params(direction = 'in',right=True)# don't put tick labels at the top
    #for axes in [ax1,ax2,ax3,ax4] : 
    #    axes.tick_params(labelbottom=False)
    for axes in [ax2,ax3,ax4] : 
        axes.tick_params(direction = 'in',right = True)# don't put tick labels at the top
        axes.tick_params(labelleft=False)

    plt.suptitle("Carbon Reservoir "+r'$M_D$',fontsize=20)
    ax1.set_xticks([1750,1950,2100])
    ax1.set_xticklabels(['1750','1950','2100'],fontsize=12)
    ax1.yaxis.set_tick_params(labelsize=13)
    ax2.set_xticks([2300,5000,7500,10000])
    ax2.set_xticklabels(['2300','5','7.5k','  10k'],fontsize=12)
    ax3.set_xticks([25000,50000,75000,100000])
    ax3.set_xticklabels(['25k','50k','75k','  100k'],fontsize=12)
    ax4.set_xticks([250000,500000])
    ax4.set_xticklabels(['250k','500k'],fontsize=12)

    ax1.set_ylabel(r'$M_D$ [PgC]',fontsize=14)

    #fig.subplots_adjust(wspace=0.05) 
    #fig.subplots_adjust(hspace=0.05) 

    ax2.set_xlabel('                            Years AD',fontsize=14)


    plt.tight_layout()
    fig.subplots_adjust(wspace=0.05) 

    fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_d.png",dpi=300)

<IPython.core.display.Javascript object>

#### 7.7 $k_{UI}, k_{IU}, k_{ID}, k_{DI}$

In [243]:
fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol_carb = solutions_CONF_B_VIC_carb_impact_param[i]
    k_ui = sol_carb[1]
    k_iu = sol_carb[2]
    k_id = sol_carb[3]
    k_di = sol_carb[4]

    sol = solutions_CONF_B_VIC[i]
    time = np.linspace(t0,tend,len(k_ui))

    ax1.plot(time+1750,k_ui,c=targetColors[i],ls='-')
    ax2.plot(time+1750,k_iu,c=targetColors[i],ls='-')
    ax3.plot(time+1750,k_id,c=targetColors[i],ls='-')
    ax4.plot(time+1750,k_di,c=targetColors[i],ls='-')

plt.suptitle('Carbon Flux Coefficients '+r'$k_{UI}, k_{IU}, k_{ID}, k_{DI}$',fontsize=22)
ax1.set_ylabel(r'$k_{UI} [s^{-1}]$',fontsize=14)
ax2.set_ylabel(r'$k_{IU} [s^{-1}]$',fontsize=14)
ax3.set_ylabel(r'$k_{ID} [s^{-1}]$',fontsize=14)
ax4.set_ylabel(r'$k_{DI} [s^{-1}]$',fontsize=14)
ax3.set_xlabel('Years AD',fontsize=14)
ax4.set_xlabel('Years AD',fontsize=14)
ax.tick_params(axis='both',labelsize=13)

#plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.6f'))

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_k_4.png", dpi=300)

<IPython.core.display.Javascript object>

#### 7.8 $k_{AU}$

In [244]:
fig,ax1 = plt.subplots(1,1,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol_carb = solutions_CONF_B_VIC_carb_impact_param[i]
    k_au = sol_carb[0]
    sol = solutions_CONF_B_VIC[i]
    time = np.linspace(t0,tend,len(k_au))

    ax1.plot(time+1750,k_au,c=targetColors[i],ls='-')


plt.suptitle('Carbon Flux Coefficient '+r'$k_{AU}$',fontsize=22)
ax1.set_ylabel(r'$k_{AU} [s^{-1}]$',fontsize=14)
ax.tick_params(axis='both',labelsize=13)
ax1.set_xlabel('Years AD',fontsize=14)

plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_carb_k_au.png", dpi=300)

<IPython.core.display.Javascript object>

### 8. $pH$

#### 8.1 $pH_U$

In [245]:
fig,ax1 = plt.subplots(1,1,figsize=(10,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_B_VIC[i]
    ph = -np.log10(species(sol,T_U,S_U,ktemp=True,level='U')[0])
    time = np.linspace(t0,tend,len(ph))

    ax1.plot(time+1750,ph,c=targetColors[i],ls='-')


plt.suptitle('Oceanic Acidity of the Upper layer '+r'$pH_U$',fontsize=22)
ax1.set_ylabel(r'$pH$',fontsize=14)
ax.tick_params(axis='both',labelsize=13)
ax1.set_xlabel('Years AD',fontsize=14)


plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_pH.png", dpi=300)

<IPython.core.display.Javascript object>

#### $pH_U, pH_I, pH_D$

In [246]:
fig,[ax1,ax2,ax3] = plt.subplots(3,1,figsize=(12,6),sharex= True)

for i in range (0, Nexp):
    sol = solutions_CONF_B_VIC[i]
    ph_u = -np.log10(species(sol,T_U,S_U,ktemp=True,level='U')[0])
    ph_i = -np.log10(species(sol,T_I,S_I,ktemp=True,level='I')[0])
    ph_d = -np.log10(species(sol,T_D,S_D,ktemp=True,level='D')[0])
    time = np.linspace(t0,tend,len(ph))

    ax1.plot(time+1750,ph_u,c=targetColors[i],ls='-')
    ax2.plot(time+1750,ph_i,c=targetColors[i],ls='-')
    ax3.plot(time+1750,ph_d,c=targetColors[i],ls='-')


plt.suptitle('Oceanic Acidity '+r'$pH_U, pH_I, pH_D$',fontsize=22)
ax1.set_ylabel(r'$pH_U$',fontsize=14)
ax2.set_ylabel(r'$pH_I$',fontsize=14)
ax3.set_ylabel(r'$pH_D$',fontsize=14)
ax.tick_params(axis='both',labelsize=13)
ax3.set_xlabel('Years AD',fontsize=14)


plt.tight_layout()

fig.savefig(save_fig_path+save_fig_path2+"CONF_B_VIC_pH_3.png", dpi=300)

<IPython.core.display.Javascript object>